In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/Shareddrives/orchid/")
!pwd

/content/drive/Shareddrives/orchid


In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

In [ ]:
name_list=[]
index=[]

image='no_background/'
path = os.path.join(image)
for i in range(len(os.listdir(path))):
  name_list.append(np.array(os.listdir(path))[i].split("Result of ")[1])

In [ ]:
#檔案名字特徵
tepal_c, tepal_p, tepal_pc, lip_c, lip_p, lip_s=np.array([]),np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
for i in range(np.shape(name_list)[0]):
  tepal_c= np.append(tepal_c, name_list[i].split("_")[0])
  tepal_p= np.append(tepal_p, name_list[i].split("_")[1])
  tepal_pc= np.append(tepal_pc, name_list[i].split("_")[2])
  lip_c= np.append(lip_c, name_list[i].split("_")[3])
  lip_p= np.append(lip_p, name_list[i].split("_")[4])
  lip_s= np.append(lip_s, name_list[i].split("_")[5])

#更改錯誤名字
tepal_pc[tepal_pc=='pimk']='pink'

In [ ]:
#6大特徵unique
df= tepal_c, tepal_p, tepal_pc, lip_c, lip_p, lip_s
ch_tepal_c= np.unique(tepal_c)
ch_tepal_p= np.unique(tepal_p)
ch_tepal_pc= np.unique(tepal_pc)
ch_lip_c= np.unique(lip_c)
ch_lip_p= np.unique(lip_p)
ch_lip_s= np.unique(lip_s)
ch_tepal_pc

array(['pink', 'purple', 'red', 'white', 'yellow'], dtype='<U32')

In [ ]:
num_df=np.zeros(shape=(6,len(name_list)))
for j in range(np.shape(df)[0]):#6
  characteristic= np.unique(df[j])
  number= np.zeros(shape=(1,len(name_list)))
  for k in range(len(characteristic)):
    number= number+k*(df[j]==characteristic[k])
  num_df[j]=number

num_df=pd.DataFrame(num_df)
num_df.set_axis(['tepal_color','tepal_pattern','tepal_pattern_color','lip_color','lip_pattern','lip_size'],
                    axis='rows', inplace=True)
num_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
tepal_color,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,4.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0
tepal_pattern,1.0,1.0,3.0,4.0,4.0,4.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,3.0,0.0,...,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,4.0,4.0,0.0,1.0,2.0,1.0,4.0
tepal_pattern_color,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,4.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0
lip_color,2.0,4.0,2.0,0.0,0.0,0.0,2.0,4.0,4.0,2.0,0.0,0.0,4.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,4.0,4.0,3.0,4.0,3.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,3.0,4.0,4.0,4.0,1.0,1.0,4.0,4.0,0.0,2.0,4.0,2.0,2.0,2.0,2.0
lip_pattern,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,0.0,1.0,4.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,5.0,0.0,1.0,5.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,5.0,0.0,1.0,0.0,1.0,0.0,1.0
lip_size,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
#6大特徵各個table
for i in range(np.shape(df)[0]):
  name= 'tepal_color', 'tepal_pattern', 'tepal_pattern_color', 'lip_color', 'lip_pattern', 'lip_size'
  characteristic= np.unique(df[i])
  count=np.array([])
  for j in range(len(characteristic)):
    count= np.append(count, np.count_nonzero(df[i] == characteristic[j]))
  percentage= np.around(count/sum(count)*100, 2)
  count= np.append(count, sum(count))
  percentage= np.append(percentage, 100)
  characteristic= np.append(characteristic, 'sum')
  data = {'characteristic':characteristic ,
      'count': count,
      'percentage': percentage}

  df1 = pd.DataFrame(data)
  print('')
  print(name[i])
  display(df1.T)


tepal_color


,0,1,2,3,4,5
characteristic,orange,pink,purple,white,yellow,sum
count,9,45,1,18,37,110
percentage,8.18,40.91,0.91,16.36,33.64,100



tepal_pattern


,0,1,2,3,4,5
characteristic,full,gradual,harlequin,spot,strip,sum
count,18,38,12,11,31,110
percentage,16.36,34.55,10.91,10,28.18,100



tepal_pattern_color


...

,0,1,2,3,4,5
characteristic,orange,purple,red,white,yellow,sum
count,10,2,58,12,28,110
percentage,9.09,1.82,52.73,10.91,25.45,100



lip_pattern


,0,1,2,3,4,5,6
characteristic,full,gradual,harlequin,spot,strip,stripe,sum
count,30,59,2,13,3,3,110
percentage,27.27,53.64,1.82,11.82,2.73,2.73,100



lip_size


,0,1,2
characteristic,big lip,normal,sum
count,4,106,110
percentage,3.64,96.36,100


In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

In [ ]:
from keras import backend as K
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):

    def precision(y_true, y_pred):
      true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
      precision = true_positives / (predicted_positives + K.epsilon())
      return precision

    def recall(y_true, y_pred):
      true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
      recall = true_positives / (possible_positives + K.epsilon())
      return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1= 2*((precision*recall)/(precision+recall+K.epsilon()))
    return f1

In [ ]:
def score(ytest, predicted_val):
    cm=confusion_matrix(ytest, predicted_val)
    acc=round(accuracy_score(ytest, predicted_val), 5)
    f1= round(f1_score(ytest, predicted_val,average='macro'), 5)
    sen= round(recall_score(ytest, predicted_val,average='macro'), 5)

    print(cm)
    print('accuracy: ', acc)
    print('f1_score: ',f1)
    print('sensitivity:', sen)
    #print('specificity:', spec)
    return cm, acc, f1, sen#, spec

分成粉紫色vs橘黃色vs白色

In [ ]:
def read_data(path, data, size):
  gray_x=[]
  rgb_x=[]
  hsv_x=[]
  lab_x=[]
  ycb_x=[]
  for p in data:
    img_array = cv2.imread(os.path.join(path,p))

    img_col=cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    new_img_array = cv2.resize(img_col, dsize=(size, size))
    gray_x.append(new_img_array)

    img_col=cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    new_img_array = cv2.resize(img_col, dsize=(size, size))
    rgb_x.append(new_img_array)

    img_col=cv2.cvtColor(img_array, cv2.COLOR_BGR2HSV)
    new_img_array = cv2.resize(img_col, dsize=(size, size))
    hsv_x.append(new_img_array)

    img_col=cv2.cvtColor(img_array, cv2.COLOR_BGR2LAB)
    new_img_array = cv2.resize(img_col, dsize=(size, size))
    lab_x.append(new_img_array)

    img_col=cv2.cvtColor(img_array, cv2.COLOR_BGR2YCR_CB)
    new_img_array = cv2.resize(img_col, dsize=(size, size))
    ycb_x.append(new_img_array)

  return(gray_x, rgb_x, hsv_x, lab_x, ycb_x)

In [ ]:
def label(num_df):
  y=[]
  for num in range(np.shape(num_df)[1]):
    if num_df[num][0]==0:#orange
      y.append(0)
    if num_df[num][0]==1:#pink
      y.append(1)
    elif num_df[num][0]==2:#purple
      y.append(1)
    elif num_df[num][0]==3:#white
      y.append(2)
    elif num_df[num][0]==4:#yellow
      y.append(0)
  return y

# read data  
color space: gray, rgb, hsv, CIELab, YCbCr  

In [ ]:
gray_y=rgb_y=hsv_y=lab_y=ycb_y=label(num_df)
gray_x, rgb_x, hsv_x, lab_x, ycb_x= read_data(path, os.listdir(path),224)
# gray_xsd= np.array(gray_x).reshape(-1, size,size,1)/255
# np.shape(gray_xsd/255)
# rgb_xsd= np.array(rgb_x)/255
gray_x=gray_x*3
gray_y=gray_y*3
rgb_x=rgb_x*3
rgb_y=rgb_y*3
hsv_x=hsv_x*3
hsv_y=hsv_y*3
lab_x=lab_x*3
lab_y=lab_y*3
ycb_x=ycb_x*3
ycb_y=ycb_y*3

# model definition  
Alexnet, vgg16  


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNet3 = Sequential()

#1st Convolutional Layer
AlexNet3.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('relu'))
AlexNet3.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet3.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('relu'))
AlexNet3.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet3.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('relu'))

#4th Convolutional Layer
AlexNet3.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('relu'))

#5th Convolutional Layer
AlexNet3.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('relu'))
AlexNet3.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet3.add(Flatten())
# 1st Fully Connected Layer
AlexNet3.add(Dense(4096, input_shape=(227,227,3,)))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet3.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet3.add(Dense(4096))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('relu'))
#Add Dropout
AlexNet3.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet3.add(Dense(1000))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('relu'))
#Add Dropout
AlexNet3.add(Dropout(0.4))

#Output Layer
AlexNet3.add(Dense(3))
AlexNet3.add(BatchNormalization())
AlexNet3.add(Activation('softmax'))
#AlexNet3.summary()
AlexNet3.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=[f1, recall, precision,'accuracy'])


In [ ]:
#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(224,224,1), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(80,80,1)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(3))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))
#AlexNet.summary()
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=[f1, recall, precision,'accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import torch
from torch import nn
from torchvision.models.vgg import vgg16
from PIL import Image
import torchvision.transforms as transforms
img_to_tensor = transforms.ToTensor()
from torch.autograd import Variable
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.models import Model

from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, Dense, Activation, Conv2DTranspose

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=8,
    min_delta=0.001,
    mode='max')

# original model  

In [ ]:
import time
def kfold(inputs, targets, batch_size, no_epochs, num_folds, verbosity, ori_model):
    start_time= time.time()
    kfold = KFold(n_splits=num_folds, shuffle=True)
    acc_per_fold =[]
    loss_per_fold =[]
    f1_per_fold =[]
    sen_per_fold =[]
    spec_per_fold =[]
    pre_per_fold=[]
    fold_no = 1
    for train, test in kfold.split(inputs, targets):
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

      # Fit data to model
        history = ori_model.fit(inputs[train], targets[train],
                  batch_size=batch_size,
                  epochs=no_epochs,callbacks=[custom_early_stopping])
              #Generate generalization metrics

        scores = ori_model.evaluate(inputs[test], targets[test], verbose=0)
        print(f'Score for fold {fold_no}: {ori_model.metrics_names} of {scores}')
        acc_per_fold.append(scores[4])
        loss_per_fold.append(scores[0])
        f1_per_fold.append(scores[1])
        sen_per_fold.append(scores[2])
        pre_per_fold.append(scores[3])
      # Increase fold number
        fold_no = fold_no + 1
        if fold_no>4:
            break
    print(acc_per_fold)
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print(f'> Fold {i+1} - f1: {f1_per_fold[i]} - sensitivity: {sen_per_fold[i]}- precision: {pre_per_fold[i]}')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    #print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    #print(f'> Loss: {np.mean(loss_per_fold)}')
    print(f'> accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> f1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
    print(f'> sensitivity: {np.mean(sen_per_fold)} (+- {np.std(sen_per_fold)})')
    print(f'> precision: {np.mean(pre_per_fold)} (+- {np.std(pre_per_fold)})')
    print(f'> specificity: {np.mean(spec_per_fold)} (+- {np.std(spec_per_fold)})')
    print('------------------------------------------------------------------------')
    end_time= time.time()
    print('time', round(end_time-start_time,2))

In [ ]:
from tensorflow.keras.utils import to_categorical
gray_x1= np.expand_dims(gray_x, axis=3)
gray_xtrain, gray_xtest, gray_ytrain, gray_ytest= train_test_split(gray_x1, gray_y, random_state=1, train_size=0.75)
gray_ytrain=to_categorical(gray_ytrain,3)

In [ ]:
ori_gray_model = Sequential()#建立一個Sequential模型，接著一層一層加上去
# Adds a densely-connected layer with 64 units to the model:
ori_gray_model.add(Conv2D(64,(3,3), activation = 'relu', input_shape =(224,224,1) ))#Convolutional Layer, 64個 feature maps
#模擬出非線性函數，簡單想像就是加入激活函數可以讓神經網路學到更多的東西．
ori_gray_model.add(MaxPooling2D(pool_size = (2,2)))
# Add another:
ori_gray_model.add(Conv2D(64,(3,3), activation = 'relu'))
ori_gray_model.add(MaxPooling2D(pool_size = (2,2)))
ori_gray_model.add(Flatten())#將 feature maps 攤平放入一個向量中
ori_gray_model.add(Dense(64, activation='relu'))

ori_gray_model.add(Dense(3, activation='sigmoid'))
ori_gray_model.compile(optimizer="adam",#用於調整參數，有做偏置校正，使每次迭代學習率都有個確定範圍，讓參數的更新較為平穩。
              loss='binary_crossentropy',#用於判斷誤差值
              metrics=[f1, precision, recall,'accuracy'])
#ori_gray_model.summary()

In [ ]:
ori_gray_model.fit(gray_xtrain, gray_ytrain, epochs=8, batch_size=32)
prediction=np.argmax(ori_gray_model.predict(np.array(gray_xtest)),axis=1)
ori_gray_cm, ori_gray_acc, ori_gray_f1, ori_gray_sen= score(gray_ytest, prediction)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/8
8/8 [==============================] - 4s 429ms/step - loss: 124.6982 - f1: 0.5321 - precision: 0.5585 - recall: 0.5401 - accuracy: 0.5830
Epoch 2/8
8/8 [==============================] - 2s 309ms/step - loss: 1.5621 - f1: 0.6180 - precision: 0.7240 - recall: 0.5552 - accuracy: 0.7652
Epoch 3/8
8/8 [==============================] - 2s 307ms/step - loss: 0.4819 - f1: 0.7849 - precision: 0.7651 - recall: 0.8118 - accuracy: 0.8340
Epoch 4/8
8/8 [==============================] - 2s 309ms/step - loss: 0.3316 - f1: 0.8828 - precision: 0.8719 - recall: 0.8947 - accuracy: 0.9109
Epoch 5/8
8/8 [==============================] - 2s 307ms/step - loss: 0.2077 - f1: 0.9266 - precision: 0.9499 - recall: 0.9056 - accuracy: 0.9312
Epoch 6/8
8/8 [==============================] - 2s 308ms/step - loss: 0.1150 - f1: 0.9642 - precision: 0.9708 - recall: 0.9579 - accuracy: 0.9636
Epoch 7/8
8/8 [==============================] - 2s 306ms/step - loss: 0.0726 - f1: 0.9766 - precision: 0.9866 - rec

In [ ]:
gray_y1=to_categorical(gray_y,2)
gray_xsd= np.array(gray_x).reshape(-1, 224,224,1)
kfold(inputs= np.array(gray_xsd), targets= np.array(gray_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=ori_gray_model)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 2s 306ms/step - loss: 0.3419 - f1: 0.9100 - precision: 0.9136 - recall: 0.9071 - accuracy: 0.8988
Epoch 2/10
8/8 [==============================] - 2s 308ms/step - loss: 0.0338 - f1: 0.9847 - precision: 0.9901 - recall: 0.9798 - accuracy: 0.9879
Epoch 3/10
8/8 [==============================] - 2s 307ms/step - loss: 0.0199 - f1: 0.9941 - precision: 0.9961 - recall: 0.9922 - accuracy: 0.9960
Epoch 4/10
8/8 [==============================] - 2s 307ms/step - loss: 0.0051 - f1: 1.0000 - precision: 1.0000 - recall: 1.0000 - accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 2s 308ms/step - loss: 0.0019 - f1: 1.0000 - precision: 1.0000 - recall: 1.0000 - accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 2s 311ms/step - loss: 6.0442e-04 - f1: 1.0000 - precision: 1.0000 - recall: 1.0000 - accuracy: 1.0000
Epoch 7/10
8/8 [==============================] - 2s 306ms/step - loss: 3.5963e-04 - f1: 1.0000 - precision: 1.0000 - r

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
rgb_xtrain, rgb_xtest, rgb_ytrain, rgb_ytest= train_test_split(rgb_x, rgb_y, random_state=1, train_size=0.75)
rgb_ytrain=to_categorical(rgb_ytrain,3)

In [ ]:
ori_rgb_model = Sequential()#建立一個Sequential模型，接著一層一層加上去
# Adds a densely-connected layer with 64 units to the model:
ori_rgb_model.add(Conv2D(64,(3,3), activation = 'relu', input_shape =(224,224,3) ))#Convolutional Layer, 64個 feature maps
#模擬出非線性函數，簡單想像就是加入激活函數可以讓神經網路學到更多的東西．
ori_rgb_model.add(MaxPooling2D(pool_size = (2,2)))
# Add another:
ori_rgb_model.add(Conv2D(64,(3,3), activation = 'relu'))
ori_rgb_model.add(MaxPooling2D(pool_size = (2,2)))
ori_rgb_model.add(Flatten())#將 feature maps 攤平放入一個向量中
ori_rgb_model.add(Dense(64, activation='relu'))

ori_rgb_model.add(Dense(3, activation='sigmoid'))
ori_rgb_model.compile(optimizer="adam",#用於調整參數，有做偏置校正，使每次迭代學習率都有個確定範圍，讓參數的更新較為平穩。
              loss='binary_crossentropy',#用於判斷誤差值
              metrics=[f1, precision, recall,'accuracy'])

In [ ]:
ori_rgb_model.fit(tf.stack(rgb_xtrain), tf.stack(rgb_ytrain), epochs=14, batch_size=16)
prediction=np.argmax(ori_rgb_model.predict(np.array(rgb_xtest)),axis=1)
ori_rgb_cm, ori_rgb_acc, ori_rgb_f1, ori_rgb_sen= score(rgb_ytest, prediction)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/14
16/16 [==============================] - 4s 202ms/step - loss: 154.1602 - f1: 0.5272 - precision: 0.5717 - recall: 0.5820 - accuracy: 0.5911
Epoch 2/14
16/16 [==============================] - 3s 174ms/step - loss: 0.6382 - f1: 0.7273 - precision: 0.7561 - recall: 0.7115 - accuracy: 0.7935
Epoch 3/14
16/16 [==============================] - 3s 175ms/step - loss: 0.3295 - f1: 0.8703 - precision: 0.8838 - recall: 0.8594 - accuracy: 0.8866
Epoch 4/14
16/16 [==============================] - 3s 175ms/step - loss: 0.2080 - f1: 0.9056 - precision: 0.9093 - recall: 0.9023 - accuracy: 0.9109
Epoch 5/14
16/16 [==============================] - 3s 176ms/step - loss: 0.2344 - f1: 0.9299 - precision: 0.9230 - recall: 0.9375 - accuracy: 0.9352
Epoch 6/14
16/16 [==============================] - 3s 176ms/step - loss: 0.1803 - f1: 0.9512 - precision: 0.9542 - recall: 0.9492 - accuracy: 0.9514
Epoch 7/14
16/16 [==============================] - 3s 175ms/step - loss: 0.1811 - f1: 0.9535 - pr

In [ ]:
tf.config.run_functions_eagerly(True)
rgb_y1=to_categorical(rgb_y,3)
kfold(inputs= np.array(rgb_x), targets= np.array(rgb_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=ori_rgb_model)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 3s 361ms/step - loss: 0.7938 - f1: 0.9145 - precision: 0.9202 - recall: 0.9095 - accuracy: 0.9190
Epoch 2/10
8/8 [==============================] - 2s 311ms/step - loss: 0.2163 - f1: 0.9316 - precision: 0.9356 - recall: 0.9282 - accuracy: 0.9352
Epoch 3/10
8/8 [==============================] - 2s 307ms/step - loss: 0.1212 - f1: 0.9773 - precision: 0.9814 - recall: 0.9735 - accuracy: 0.9798
Epoch 4/10
8/8 [==============================] - 2s 309ms/step - loss: 0.0931 - f1: 0.9791 - precision: 0.9796 - recall: 0.9789 - accuracy: 0.9798
Epoch 5/10
8/8 [==============================] - 2s 310ms/step - loss: 0.0666 - f1: 0.9799 - precision: 0.9755 - recall: 0.9844 - accuracy: 0.9838
Epoch 6/10
8/8 [==============================] - 2s 307ms/step - loss: 0.0541 - f1: 0.9883 - precision: 0.9883 - recall: 0.9883 - accuracy: 0.9879
Epoch 7/10
8/8 [==============================] - 2s 307ms/step - loss: 0.0407 - f1: 0.9941 - precision: 0.9961 - recall: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
hsv_xtrain, hsv_xtest, hsv_ytrain, hsv_ytest= train_test_split(hsv_x, hsv_y, random_state=1, train_size=0.75)
hsv_ytrain=to_categorical(hsv_ytrain, 3)

In [ ]:
ori_rgb_model.fit(tf.stack(hsv_xtrain), tf.stack(hsv_ytrain), epochs=8, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(ori_rgb_model.predict(np.array(hsv_xtest)),axis=1)
ori_hsv_cm, ori_hsv_acc, ori_hsv_f1, ori_hsv_sen= score(hsv_ytest, prediction)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/8
7/7 [==============================] - 4s 348ms/step - loss: 1.2203 - f1: 0.7995 - precision: 0.8026 - recall: 0.7985 - accuracy: 0.8057
Epoch 2/8
7/7 [==============================] - 2s 346ms/step - loss: 0.3436 - f1: 0.8539 - precision: 0.8630 - recall: 0.8474 - accuracy: 0.8462
Epoch 3/8
7/7 [==============================] - 2s 350ms/step - loss: 0.2274 - f1: 0.8978 - precision: 0.9144 - recall: 0.8821 - accuracy: 0.8988
Epoch 4/8
7/7 [==============================] - 2s 347ms/step - loss: 0.1439 - f1: 0.9517 - precision: 0.9539 - recall: 0.9500 - accuracy: 0.9555
Epoch 5/8
7/7 [==============================] - 2s 347ms/step - loss: 0.1167 - f1: 0.9584 - precision: 0.9634 - recall: 0.9536 - accuracy: 0.9555
Epoch 6/8
7/7 [==============================] - 2s 349ms/step - loss: 0.0589 - f1: 0.9838 - precision: 0.9855 - recall: 0.9821 - accuracy: 0.9838
Epoch 7/8
7/7 [==============================] - 3s 351ms/step - loss: 0.0346 - f1: 0.9910 - precision: 0.9929 - recal

In [ ]:
hsv_y1=to_categorical(hsv_y,3)
kfold(inputs= np.array(hsv_x), targets= np.array(hsv_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=ori_rgb_model)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 2s 308ms/step - loss: 0.2395 - f1: 0.9497 - precision: 0.9458 - recall: 0.9540 - accuracy: 0.9595
Epoch 2/10
8/8 [==============================] - 2s 307ms/step - loss: 0.0859 - f1: 0.9736 - precision: 0.9789 - recall: 0.9688 - accuracy: 0.9838
Epoch 3/10
8/8 [==============================] - 2s 309ms/step - loss: 0.1067 - f1: 0.9729 - precision: 0.9657 - recall: 0.9805 - accuracy: 0.9717
Epoch 4/10
8/8 [==============================] - 2s 309ms/step - loss: 0.0243 - f1: 0.9961 - precision: 0.9961 - recall: 0.9961 - accuracy: 0.9960
Epoch 5/10
8/8 [==============================] - 2s 309ms/step - loss: 0.0043 - f1: 1.0000 - precision: 1.0000 - recall: 1.0000 - accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 2s 310ms/step - loss: 0.0036 - f1: 1.0000 - precision: 1.0000 - recall: 1.0000 - accuracy: 1.0000
Epoch 7/10
8/8 [==============================] - 2s 309ms/step - loss: 0.0023 - f1: 1.0000 - precision: 1.0000 - recall: 1

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
lab_xtrain, lab_xtest, lab_ytrain, lab_ytest= train_test_split(lab_x, lab_y, random_state=1, train_size=0.75)
lab_ytrain=to_categorical(lab_ytrain, 3)

ori_rgb_model.fit(tf.stack(lab_xtrain), tf.stack(lab_ytrain), epochs=30, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(ori_rgb_model.predict(np.array(lab_xtest)),axis=1)
ori_lab_cm, ori_lab_acc, ori_lab_f1, ori_lab_sen= score(lab_ytest, prediction)

In [ ]:
lab_y1=to_categorical(lab_y,3)
kfold(inputs= np.array(lab_x), targets= np.array(lab_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=ori_rgb_model)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 2s 310ms/step - loss: 0.6463 - f1: 0.6534 - precision: 0.6534 - recall: 0.6534 - accuracy: 0.6518
Epoch 2/10
8/8 [==============================] - 2s 308ms/step - loss: 0.6463 - f1: 0.6503 - precision: 0.6503 - recall: 0.6503 - accuracy: 0.6518
Epoch 3/10
8/8 [==============================] - 2s 308ms/step - loss: 0.6463 - f1: 0.6488 - precision: 0.6488 - recall: 0.6488 - accuracy: 0.6518
Epoch 4/10
8/8 [==============================] - 2s 308ms/step - loss: 0.6462 - f1: 0.6534 - precision: 0.6534 - recall: 0.6534 - accuracy: 0.6518
Epoch 5/10
8/8 [==============================] - 2s 309ms/step - loss: 0.6462 - f1: 0.6488 - precision: 0.6488 - recall: 0.6488 - accuracy: 0.6518
Epoch 6/10
8/8 [==============================] - 2s 308ms/step - loss: 0.6461 - f1: 0.6518 - precision: 0.6518 - recall: 0.6518 - accuracy: 0.6518
Epoch 7/10
8/8 [==============================] - 2s 309ms/step - loss: 0.6461 - f1: 0.6534 - precision: 0.6534 - recall: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
ycb_xtrain, ycb_xtest, ycb_ytrain, ycb_ytest= train_test_split(ycb_x, ycb_y, random_state=1, train_size=0.75)
ycb_ytrain=to_categorical(ycb_ytrain, 3)

ori_rgb_model.fit(tf.stack(ycb_xtrain), tf.stack(ycb_ytrain), epochs=30, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(ori_rgb_model.predict(np.array(ycb_xtest)),axis=1)
ori_ycb_cm, ori_ycb_acc, ori_ycb_f1, ori_ycb_sen= score(ycb_ytest, prediction)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
7/7 [==============================] - 3s 345ms/step - loss: 0.6514 - f1: 0.6520 - precision: 0.6520 - recall: 0.6520 - accuracy: 0.6437
Epoch 2/30
7/7 [==============================] - 2s 346ms/step - loss: 0.6513 - f1: 0.6689 - precision: 0.6689 - recall: 0.6689 - accuracy: 0.6437
Epoch 3/30
7/7 [==============================] - 2s 346ms/step - loss: 0.6513 - f1: 0.6520 - precision: 0.6520 - recall: 0.6520 - accuracy: 0.6437
Epoch 4/30
7/7 [==============================] - 2s 347ms/step - loss: 0.6513 - f1: 0.6857 - precision: 0.6857 - recall: 0.6857 - accuracy: 0.6437
Epoch 5/30
7/7 [==============================] - 2s 346ms/step - loss: 0.6513 - f1: 0.6184 - precision: 0.6184 - recall: 0.6184 - accuracy: 0.6437
Epoch 6/30
7/7 [==============================] - 2s 346ms/step - loss: 0.6514 - f1: 0.6520 - precision: 0.6520 - recall: 0.6520 - accuracy: 0.6437
Epoch 7/30
7/7 [==============================] - 2s 346ms/step - loss: 0.6514 - f1: 0.6520 - precision: 0.6520 

In [ ]:
ycb_y1=to_categorical(ycb_y,3)
kfold(inputs= np.array(ycb_x), targets= np.array(ycb_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=ori_rgb_model)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 2s 307ms/step - loss: 0.6464 - f1: 0.6534 - precision: 0.6534 - recall: 0.6534 - accuracy: 0.6518
Epoch 2/10
8/8 [==============================] - 2s 311ms/step - loss: 0.6463 - f1: 0.6518 - precision: 0.6518 - recall: 0.6518 - accuracy: 0.6518
Epoch 3/10
8/8 [==============================] - 2s 307ms/step - loss: 0.6463 - f1: 0.6549 - precision: 0.6549 - recall: 0.6549 - accuracy: 0.6518
Epoch 4/10
8/8 [==============================] - 2s 308ms/step - loss: 0.6464 - f1: 0.6488 - precision: 0.6488 - recall: 0.6488 - accuracy: 0.6518
Epoch 5/10
8/8 [==============================] - 2s 308ms/step - loss: 0.6464 - f1: 0.6503 - precision: 0.6503 - recall: 0.6503 - accuracy: 0.6518
Epoch 6/10
8/8 [==============================] - 2s 308ms/step - loss: 0.6463 - f1: 0.6518 - precision: 0.6518 - recall: 0.6518 - accuracy: 0.6518
Epoch 7/10
8/8 [==============================] - 2s 308ms/step - loss: 0.6463 - f1: 0.6488 - precision: 0.6488 - recall: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#alexnet  

In [ ]:
AlexNet.fit(gray_xtrain,gray_ytrain, batch_size=10, epochs=30,callbacks=[custom_early_stopping])
pred=np.argmax(AlexNet.predict(gray_xtest),axis=1)
alex_gray_cm, alex_gray_acc, alex_gray_f1, alex_gray_sen= score(gray_ytest, pred)

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


25/25 [==============================] - 4s 135ms/step - loss: 0.8261 - f1: 0.5429 - recall: 0.5429 - precision: 0.5429 - accuracy: 0.5425
Epoch 2/30
25/25 [==============================] - 3s 112ms/step - loss: 0.6597 - f1: 0.6331 - recall: 0.6331 - precision: 0.6331 - accuracy: 0.6356
Epoch 3/30
25/25 [==============================] - 3s 113ms/step - loss: 0.6512 - f1: 0.6343 - recall: 0.6343 - precision: 0.6343 - accuracy: 0.6316
Epoch 4/30
25/25 [==============================] - 3s 116ms/step - loss: 0.6284 - f1: 0.6366 - recall: 0.6366 - precision: 0.6366 - accuracy: 0.6356
Epoch 5/30
25/25 [==============================] - 3s 114ms/step - loss: 0.6102 - f1: 0.6686 - recall: 0.6686 - precision: 0.6686 - accuracy: 0.6680
Epoch 6/30
25/25 [==============================] - 3s 114ms/step - loss: 0.5527 - f1: 0.7383 - recall: 0.7383 - precision: 0.7383 - accuracy: 0.7368
Epoch 7/30
25/25 [==============================] - 3s 113ms/step - loss: 0.6188 - f1: 0.6663 - recall: 0.6663 

In [ ]:
AlexNet3.fit(tf.stack(rgb_xtrain),tf.stack(rgb_ytrain), batch_size=20, epochs=30,callbacks=[custom_early_stopping])
pred=np.argmax(AlexNet3.predict(np.array(rgb_xtest)),axis=1)
alex_rgb_cm, alex_rgb_acc, alex_rgb_f1, alex_rgb_sen= score(rgb_ytest, pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
13/13 [==============================] - 3s 150ms/step - loss: 0.8721 - f1: 0.5357 - recall: 0.5357 - precision: 0.5357 - accuracy: 0.5263
Epoch 2/30
13/13 [==============================] - 2s 144ms/step - loss: 0.7832 - f1: 0.5709 - recall: 0.5709 - precision: 0.5709 - accuracy: 0.5709
Epoch 3/30
13/13 [==============================] - 2s 144ms/step - loss: 0.6611 - f1: 0.6363 - recall: 0.6363 - precision: 0.6363 - accuracy: 0.6397
Epoch 4/30
13/13 [==============================] - 2s 145ms/step - loss: 0.5939 - f1: 0.7236 - recall: 0.7236 - precision: 0.7236 - accuracy: 0.7166
Epoch 5/30
13/13 [==============================] - 2s 144ms/step - loss: 0.5506 - f1: 0.7511 - recall: 0.7511 - precision: 0.7511 - accuracy: 0.7530
Epoch 6/30
13/13 [==============================] - 2s 144ms/step - loss: 0.5892 - f1: 0.7016 - recall: 0.7016 - precision: 0.7016 - accuracy: 0.7085
Epoch 7/30
13/13 [==============================] - 2s 145ms/step - loss: 0.5011 - f1: 0.7478 - reca

In [ ]:
AlexNet3.fit(tf.stack(hsv_xtrain),tf.stack(hsv_ytrain), batch_size=10, epochs=50,callbacks=[custom_early_stopping])
pred=np.argmax(AlexNet3.predict(np.array(hsv_xtest)),axis=1)
alex_hsv_cm, alex_hsv_acc, alex_hsv_f1, alex_hsv_sen= score(hsv_ytest, pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/50
25/25 [==============================] - 3s 120ms/step - loss: 0.5741 - f1: 0.7663 - recall: 0.7663 - precision: 0.7663 - accuracy: 0.7652
Epoch 2/50
25/25 [==============================] - 3s 119ms/step - loss: 0.4653 - f1: 0.8223 - recall: 0.8223 - precision: 0.8223 - accuracy: 0.8219
Epoch 3/50
25/25 [==============================] - 3s 119ms/step - loss: 0.3991 - f1: 0.8240 - recall: 0.8240 - precision: 0.8240 - accuracy: 0.8219
Epoch 4/50
25/25 [==============================] - 3s 127ms/step - loss: 0.3987 - f1: 0.8320 - recall: 0.8320 - precision: 0.8320 - accuracy: 0.8300
Epoch 5/50
25/25 [==============================] - 3s 120ms/step - loss: 0.2994 - f1: 0.8720 - recall: 0.8720 - precision: 0.8720 - accuracy: 0.8704
Epoch 6/50
25/25 [==============================] - 3s 119ms/step - loss: 0.2550 - f1: 0.9280 - recall: 0.9280 - precision: 0.9280 - accuracy: 0.9271
Epoch 7/50
25/25 [==============================] - 3s 121ms/step - loss: 0.2967 - f1: 0.8760 - reca

In [ ]:
AlexNet3.fit(tf.stack(lab_xtrain), tf.stack(lab_ytrain), epochs=30, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(AlexNet3.predict(np.array(lab_xtest)),axis=1)
alex_lab_cm, alex_lab_acc, alex_lab_f1, alex_lab_sen= score(lab_ytest, prediction)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
7/7 [==============================] - 2s 194ms/step - loss: 0.6367 - f1: 0.7520 - recall: 0.7520 - precision: 0.7520 - accuracy: 0.7571
Epoch 2/30
7/7 [==============================] - 1s 190ms/step - loss: 0.1625 - f1: 0.9357 - recall: 0.9357 - precision: 0.9357 - accuracy: 0.9271
Epoch 3/30
7/7 [==============================] - 1s 187ms/step - loss: 0.1016 - f1: 0.9617 - recall: 0.9617 - precision: 0.9617 - accuracy: 0.9757
Epoch 4/30
7/7 [==============================] - 1s 187ms/step - loss: 0.0760 - f1: 0.9929 - recall: 0.9929 - precision: 0.9929 - accuracy: 0.9919
Epoch 5/30
7/7 [==============================] - 1s 189ms/step - loss: 0.0613 - f1: 0.9964 - recall: 0.9964 - precision: 0.9964 - accuracy: 0.9960
Epoch 6/30
7/7 [==============================] - 1s 187ms/step - loss: 0.0561 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 7/30
7/7 [==============================] - 1s 193ms/step - loss: 0.0524 - f1: 1.0000 - recall: 1.0000 - p

In [ ]:
AlexNet3.fit(tf.stack(ycb_xtrain), tf.stack(ycb_ytrain), epochs=30, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(AlexNet3.predict(np.array(ycb_xtest)),axis=1)
alex_ycb_cm, alex_ycb_acc, alex_ycb_f1, alex_ycb_sen= score(ycb_ytest, prediction)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
7/7 [==============================] - 1s 198ms/step - loss: 0.0898 - f1: 0.9821 - recall: 0.9821 - precision: 0.9821 - accuracy: 0.9798
Epoch 2/30
7/7 [==============================] - 1s 190ms/step - loss: 0.0547 - f1: 0.9760 - recall: 0.9760 - precision: 0.9760 - accuracy: 0.9919
Epoch 3/30
7/7 [==============================] - 1s 190ms/step - loss: 0.0479 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 4/30
7/7 [==============================] - 1s 188ms/step - loss: 0.0579 - f1: 0.9964 - recall: 0.9964 - precision: 0.9964 - accuracy: 0.9960
Epoch 5/30
7/7 [==============================] - 1s 187ms/step - loss: 0.0504 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 6/30
7/7 [==============================] - 1s 188ms/step - loss: 0.0453 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 7/30
7/7 [==============================] - 1s 191ms/step - loss: 0.0418 - f1: 1.0000 - recall: 1.0000 - p

In [ ]:
def alex_kfold(inputs, targets, batch_size, no_epochs, num_folds, verbosity, alex):
    kfold = KFold(n_splits=num_folds, shuffle=True)
    acc_per_fold =[]
    loss_per_fold =[]
    f1_per_fold =[]
    sen_per_fold =[]
    spec_per_fold =[]
    fold_no = 1
    targets=to_categorical(targets)
    for train, test in kfold.split(inputs, targets):

        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

      # Fit data to model
        history = alex.fit(inputs[train], targets[train],
                  batch_size=batch_size,
                  epochs=no_epochs,callbacks=[custom_early_stopping])

      #Generate generalization metrics
        scores = alex.evaluate(inputs[test], targets[test], verbose=0)
        print(scores)
        print(f'Score for fold {fold_no}: {alex.metrics_names} of {scores}')
        acc_per_fold.append(scores[4])
        loss_per_fold.append(scores[0])
        f1_per_fold.append(scores[1])
        sen_per_fold.append(scores[2])

      # Increase fold number
        fold_no = fold_no + 1
        if fold_no>4:
            break
    print(acc_per_fold)
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print(f'> Fold {i+1} - f1: {f1_per_fold[i]} - sensitivity: {sen_per_fold[i]}')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    #print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    #print(f'> Loss: {np.mean(loss_per_fold)}')
    print(f'> accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> f1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
    print(f'> sensitivity: {np.mean(sen_per_fold)} (+- {np.std(sen_per_fold)})')
    print(f'> specificity: {np.mean(spec_per_fold)} (+- {np.std(spec_per_fold)})')
    print('------------------------------------------------------------------------')

In [ ]:
gray_xsd= np.array(gray_x).reshape(-1, 224,224,1)
alex_kfold(inputs= np.array(gray_xsd), targets= np.array(gray_y), batch_size=32, no_epochs=10, num_folds=4, verbosity=1,alex=AlexNet)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 2s 252ms/step - loss: 0.2610 - f1: 0.9001 - recall: 0.9001 - precision: 0.9001 - accuracy: 0.9028
Epoch 2/10
8/8 [==============================] - 1s 155ms/step - loss: 0.1922 - f1: 0.9516 - recall: 0.9516 - precision: 0.9516 - accuracy: 0.9514
Epoch 3/10
8/8 [==============================] - 1s 157ms/step - loss: 0.1635 - f1: 0.9727 - recall: 0.9727 - precision: 0.9727 - accuracy: 0.9717
Epoch 4/10
8/8 [==============================] - 1s 156ms/step - loss: 0.1573 - f1: 0.9750 - recall: 0.9750 - precision: 0.9750 - accuracy: 0.9757
Epoch 5/10
8/8 [==============================] - 1s 155ms/step - loss: 0.1338 - f1: 0.9922 - recall: 0.9922 - precision: 0.9922 - accuracy: 0.9919
Epoch 6/10
8/8 [==============================] - 1s 155ms/step - loss: 0.1314 - f1: 0.9883 - recall: 0.9883 - precision: 0.9883 - accuracy: 0.9879
Epoch 7/10
8/8 [==============================] - 1s 155ms/step - loss: 0.1133 - f1: 0.9961 - recall: 0.9961 - precision: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
alex_kfold(inputs= np.array(rgb_x), targets= np.array(rgb_y), batch_size=32, no_epochs=10, num_folds=4, verbosity=1,alex=AlexNet3)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 2s 182ms/step - loss: 0.2432 - f1: 0.9158 - recall: 0.9158 - precision: 0.9158 - accuracy: 0.9190
Epoch 2/10
8/8 [==============================] - 1s 171ms/step - loss: 0.1425 - f1: 0.9609 - recall: 0.9609 - precision: 0.9609 - accuracy: 0.9595
Epoch 3/10
8/8 [==============================] - 1s 170ms/step - loss: 0.0984 - f1: 0.9828 - recall: 0.9828 - precision: 0.9828 - accuracy: 0.9838
Epoch 4/10
8/8 [==============================] - 1s 169ms/step - loss: 0.1217 - f1: 0.9750 - recall: 0.9750 - precision: 0.9750 - accuracy: 0.9757
Epoch 5/10
8/8 [==============================] - 1s 170ms/step - loss: 0.0659 - f1: 0.9922 - recall: 0.9922 - precision: 0.9922 - accuracy: 0.9919
Epoch 6/10
8/8 [==============================] - 1s 171ms/step - loss: 0.0780 - f1: 0.9844 - recall: 0.9844 - precision: 0.9844 - accuracy: 0.9838
Epoch 7/10
8/8 [==============================] - 1s 169ms/step - loss: 0.0590 - f1: 0.9883 - recall: 0.9883 - precision: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
alex_kfold(inputs= np.array(hsv_x), targets= np.array(hsv_y), batch_size=32, no_epochs=10, num_folds=4, verbosity=1,alex=AlexNet3)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 1s 170ms/step - loss: 0.4284 - f1: 0.8618 - recall: 0.8618 - precision: 0.8618 - accuracy: 0.8583
Epoch 2/10
8/8 [==============================] - 1s 172ms/step - loss: 0.1007 - f1: 0.9633 - recall: 0.9633 - precision: 0.9633 - accuracy: 0.9636
Epoch 3/10
8/8 [==============================] - 1s 171ms/step - loss: 0.0615 - f1: 0.9789 - recall: 0.9789 - precision: 0.9789 - accuracy: 0.9798
Epoch 4/10
8/8 [==============================] - 1s 172ms/step - loss: 0.0641 - f1: 0.9766 - recall: 0.9766 - precision: 0.9766 - accuracy: 0.9757
Epoch 5/10
8/8 [==============================] - 1s 171ms/step - loss: 0.0343 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 1s 172ms/step - loss: 0.0429 - f1: 0.9922 - recall: 0.9922 - precision: 0.9922 - accuracy: 0.9919
Epoch 7/10
8/8 [==============================] - 1s 171ms/step - loss: 0.0389 - f1: 0.9946 - recall: 0.9946 - precision: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
alex_kfold(inputs= np.array(lab_x), targets= np.array(lab_y), batch_size=32, no_epochs=10, num_folds=4, verbosity=1,alex=AlexNet3)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 1s 170ms/step - loss: 0.5463 - f1: 0.8133 - recall: 0.8133 - precision: 0.8133 - accuracy: 0.8097
Epoch 2/10
8/8 [==============================] - 1s 170ms/step - loss: 0.1422 - f1: 0.9344 - recall: 0.9344 - precision: 0.9344 - accuracy: 0.9352
Epoch 3/10
8/8 [==============================] - 1s 170ms/step - loss: 0.0890 - f1: 0.9711 - recall: 0.9711 - precision: 0.9711 - accuracy: 0.9717
Epoch 4/10
8/8 [==============================] - 1s 171ms/step - loss: 0.0387 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 1s 170ms/step - loss: 0.0377 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 1s 172ms/step - loss: 0.0400 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 7/10
8/8 [==============================] - 1s 170ms/step - loss: 0.0446 - f1: 0.9922 - recall: 0.9922 - precision: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
alex_kfold(inputs= np.array(ycb_x), targets= np.array(ycb_y), batch_size=32, no_epochs=10, num_folds=4, verbosity=1,alex=AlexNet3)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 1s 172ms/step - loss: 0.0253 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 2/10
8/8 [==============================] - 1s 169ms/step - loss: 0.0271 - f1: 0.9961 - recall: 0.9961 - precision: 0.9961 - accuracy: 0.9960
Epoch 3/10
8/8 [==============================] - 1s 169ms/step - loss: 0.0194 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 4/10
8/8 [==============================] - 1s 171ms/step - loss: 0.0208 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 1s 171ms/step - loss: 0.0242 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 1s 168ms/step - loss: 0.0180 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 7/10
8/8 [==============================] - 1s 169ms/step - loss: 0.0224 - f1: 1.0000 - recall: 1.0000 - precision: 1

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#vgg16  

In [ ]:
def vgg_kfold(inputs, targets, batch_size, no_epochs, num_folds, verbosity):
    start_time=time.time()
    kfold = KFold(n_splits=num_folds, shuffle=True)
    acc_per_fold =[]
    loss_per_fold =[]
    f1_per_fold =[]
    sen_per_fold =[]
    spec_per_fold =[]
    pre_per_fold=[]

    fold_no = 1
    for train, test in kfold.split(inputs, targets):

        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

      # Fit data to model
        history = model.fit(inputs[train], targets[train],
                  batch_size=batch_size,
                  epochs=no_epochs)

      #Generate generalization metrics
        scores = model.evaluate(inputs[test], targets[test], verbose=0)
        print(scores)
        print(f'Score for fold {fold_no}: {model.metrics_names} of {scores}')
        acc_per_fold.append(scores[4])
        loss_per_fold.append(scores[0])
        f1_per_fold.append(scores[1])
        sen_per_fold.append(scores[2])
        pre_per_fold.append(scores[3])

      # Increase fold number
        fold_no = fold_no + 1
        if fold_no>4:
            break
    print(acc_per_fold)
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print(f'> Fold {i+1} - f1: {f1_per_fold[i]} - sensitivity: {sen_per_fold[i]}-precision:{pre_per_fold[i]}')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    #print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    #print(f'> Loss: {np.mean(loss_per_fold)}')
    print(f'> accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> f1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
    print(f'> precision: {np.mean(pre_per_fold)} (+- {np.std(pre_per_fold)})')
    print(f'> sensitivity: {np.mean(sen_per_fold)} (+- {np.std(sen_per_fold)})')
    print(f'> specificity: {np.mean(spec_per_fold)} (+- {np.std(spec_per_fold)})')
    print('------------------------------------------------------------------------')
    end_time=time.time()
    print('time', end_time-start_time)

In [ ]:
# input_shape = (224, 224, 1)

# model = Sequential([
#     Conv2D(64, (3, 3), input_shape=input_shape, padding='same',
#            activation='relu'),
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     Conv2D(128, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Flatten(),
#     Dense(4096, activation='relu'),
#     Dense(4096, activation='relu'),
#     Dense(2, activation='softmax')
# ])

# sgd = SGD(lr=1e-8, momentum=0.9)
# model.compile(optimizer = sgd,
#               loss='binary_crossentropy',
#               metrics = [f1, precision, recall,'accuracy'])

In [ ]:
#VGG16预训练网络
covn_base = tf.keras.applications.vgg16.VGG16(weights='imagenet',include_top=False)
covn_base.trainable = True
#冻结前面的层，训练最后四层
for layers in covn_base.layers[:-4]:
    layers.trainable = False
#构建模型
model = tf.keras.Sequential()
model.add(covn_base)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.summary()
#编译模型，初始学习率0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[f1, precision, recall,'accuracy'])
#监视'val_loss'，当两个epoch不变时，学习率减小为1/10
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 514       
Total params: 14,846,530
Trainable params: 7,211,266
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
from tensorflow.keras.utils import to_categorical

#gray_x, y=read_image(colortype=cv2.IMREAD_GRAYSCALE, size=224)
a=np.array([gray_x,gray_x,gray_x])
gray_x2= a.transpose(1,2,3,0)

gray_x1= tf.keras.applications.vgg16.preprocess_input(np.array(gray_x2))
gray_xtrain1, gray_xtest1, gray_ytrain, gray_ytest= train_test_split(gray_x1, gray_y, random_state=1, train_size=0.75)
gray_ytrain=to_categorical(gray_ytrain,2)

In [ ]:
tf.config.run_functions_eagerly(True)

model.fit(gray_xtrain1, gray_ytrain, epochs=15, validation_split=0.2, callbacks=[custom_early_stopping])
pred=np.argmax(model.predict(np.array(gray_xtest1)),axis=1)
vgg_gray_cm, vgg_gray_acc, vgg_gray_f1, vgg_gray_sen= score(gray_ytest, pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/15
7/7 [==============================] - 9s 1s/step - loss: 19.9608 - f1: 0.4473 - precision: 0.4473 - recall: 0.4473 - accuracy: 0.4264 - val_loss: 15.6136 - val_f1: 0.3264 - val_precision: 0.3264 - val_recall: 0.3264 - val_accuracy: 0.3400
Epoch 2/15
7/7 [==============================] - 9s 1s/step - loss: 13.7198 - f1: 0.5321 - precision: 0.5321 - recall: 0.5321 - accuracy: 0.5228 - val_loss: 14.1948 - val_f1: 0.3264 - val_precision: 0.3264 - val_recall: 0.3264 - val_accuracy: 0.3400
Epoch 3/15
7/7 [==============================] - 9s 1s/step - loss: 15.2929 - f1: 0.5080 - precision: 0.5080 - recall: 0.5080 - accuracy: 0.5228 - val_loss: 0.6555 - val_f1: 0.5781 - val_precision: 0.5781 - val_recall: 0.5781 - val_accuracy: 0.6000
Epoch 4/15
7/7 [==============================] - 8s 1s/step - loss: 0.8029 - f1: 0.5170 - precision: 0.5170 - recall: 0.5170 - accuracy: 0.5330 - val_loss: 0.6790 - val_f1: 0.6580 - val_precision: 0.6580 - val_recall: 0.6580 - val_accuracy: 0.6400

In [ ]:
gray_y1=to_categorical(gray_y,2)
kfold(inputs= np.array(gray_x1), targets= np.array(gray_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=model)

------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/10
8/8 [==============================] - 9s 1s/step - loss: 0.6469 - f1: 0.6605 - precision: 0.6605 - recall: 0.6605 - accuracy: 0.6640
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.6447 - f1: 0.6620 - precision: 0.6620 - recall: 0.6620 - accuracy: 0.6640
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.6437 - f1: 0.6681 - precision: 0.6681 - recall: 0.6681 - accuracy: 0.6640
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.6426 - f1: 0.6590 - precision: 0.6590 - recall: 0.6590 - accuracy: 0.6640
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.6393 - f1: 0.6681 - precision: 0.6681 - recall: 0.6681 - accuracy: 0.6640
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.6408 - f1: 0.6636 - precision: 0.6636 - recall: 0.6636 - accuracy: 0.6640
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.6394 - f1: 0.6666 - precision: 0.6666 - recall: 0.6666 - ac

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# input_shape = (224, 224, 3)

# model = Sequential([
#     Conv2D(64, (3, 3), input_shape=input_shape, padding='same',
#            activation='relu'),
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     Conv2D(128, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Flatten(),
#     Dense(4096, activation='relu'),
#     Dense(4096, activation='relu'),
#     Dense(2, activation='softmax')
# ])

# sgd = SGD(lr=1e-8, momentum=0.9)
# model.compile(optimizer = sgd,
#               loss='binary_crossentropy',
#               metrics = [f1, precision, recall,'accuracy'])
# model.summary()

In [ ]:
from tensorflow.keras.utils import to_categorical
rgb_x1= tf.keras.applications.vgg16.preprocess_input(np.array(rgb_x))
rgb_xtrain1, rgb_xtest1, rgb_ytrain, rgb_ytest= train_test_split(rgb_x1, rgb_y, random_state=1, train_size=0.75)
rgb_ytrain=to_categorical(rgb_ytrain,2)

tf.config.run_functions_eagerly(True)

model.fit(rgb_xtrain1, rgb_ytrain, epochs=30, validation_split=0.2, verbose=2, callbacks=[custom_early_stopping])
pred=np.argmax(model.predict(np.array(rgb_xtest1)),axis=1)
vgg_rgb_cm, vgg_rgb_acc, vgg_rgb_f1, vgg_rgb_sen= score(rgb_ytest, pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
7/7 - 9s - loss: 0.6579 - f1: 0.6348 - precision: 0.6348 - recall: 0.6348 - accuracy: 0.6396 - val_loss: 0.6417 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 2/30
7/7 - 9s - loss: 0.6575 - f1: 0.6348 - precision: 0.6348 - recall: 0.6348 - accuracy: 0.6396 - val_loss: 0.6415 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 3/30
7/7 - 9s - loss: 0.6556 - f1: 0.5625 - precision: 0.5625 - recall: 0.5625 - accuracy: 0.6396 - val_loss: 0.6412 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 4/30
7/7 - 8s - loss: 0.6543 - f1: 0.6348 - precision: 0.6348 - recall: 0.6348 - accuracy: 0.6396 - val_loss: 0.6411 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 5/30
7/7 - 9s - loss: 0.6554 - f1: 0.6348 - precision: 0.6348 - recall: 0.6348 - accuracy: 0.6396 - val_loss: 0.6414 - val_f1: 0.6736 - val_precision: 0.6736 - va

In [ ]:
from tensorflow.keras.utils import to_categorical
hsv_x1= tf.keras.applications.vgg16.preprocess_input(np.array(hsv_x))
hsv_xtrain1, hsv_xtest1, hsv_ytrain, hsv_ytest= train_test_split(hsv_x1, hsv_y, random_state=1, train_size=0.75)
hsv_ytrain=to_categorical(hsv_ytrain,2)

from tensorflow.keras.callbacks import EarlyStopping

model.fit(hsv_xtrain1, hsv_ytrain, epochs=30, validation_split=0.2, verbose=2, callbacks=[custom_early_stopping])
pred=np.argmax(model.predict(np.array(hsv_xtest1)),axis=1)
vgg_hsv_cm, vgg_hsv_acc, vgg_hsv_f1, vgg_hsv_sen= score(hsv_ytest, pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
7/7 - 9s - loss: 0.6531 - f1: 0.6107 - precision: 0.6107 - recall: 0.6107 - accuracy: 0.6396 - val_loss: 0.6424 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 2/30
7/7 - 8s - loss: 0.6522 - f1: 0.6348 - precision: 0.6348 - recall: 0.6348 - accuracy: 0.6396 - val_loss: 0.6429 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 3/30
7/7 - 9s - loss: 0.6517 - f1: 0.6589 - precision: 0.6589 - recall: 0.6589 - accuracy: 0.6396 - val_loss: 0.6429 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 4/30
7/7 - 9s - loss: 0.6537 - f1: 0.6589 - precision: 0.6589 - recall: 0.6589 - accuracy: 0.6396 - val_loss: 0.6426 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 5/30
7/7 - 8s - loss: 0.6527 - f1: 0.6830 - precision: 0.6830 - recall: 0.6830 - accuracy: 0.6396 - val_loss: 0.6422 - val_f1: 0.6736 - val_precision: 0.6736 - va

In [ ]:
from tensorflow.keras.utils import to_categorical
lab_x1= tf.keras.applications.vgg16.preprocess_input(np.array(lab_x))
lab_xtrain1, lab_xtest1, lab_ytrain, lab_ytest= train_test_split(lab_x1, lab_y, random_state=1, train_size=0.75)
lab_ytrain=to_categorical(lab_ytrain,2)

from tensorflow.keras.callbacks import EarlyStopping

model.fit(lab_xtrain1, lab_ytrain, epochs=30, validation_split=0.2, verbose=2, callbacks=[custom_early_stopping])
pred=np.argmax(model.predict(np.array(lab_xtest1)),axis=1)
vgg_lab_cm, vgg_lab_acc, vgg_lab_f1, vgg_lab_sen= score(lab_ytest, pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
7/7 - 8s - loss: 0.6536 - f1: 0.6107 - precision: 0.6107 - recall: 0.6107 - accuracy: 0.6396 - val_loss: 0.6418 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 2/30
7/7 - 9s - loss: 0.6547 - f1: 0.6589 - precision: 0.6589 - recall: 0.6589 - accuracy: 0.6396 - val_loss: 0.6423 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 3/30
7/7 - 8s - loss: 0.6544 - f1: 0.6348 - precision: 0.6348 - recall: 0.6348 - accuracy: 0.6396 - val_loss: 0.6424 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 4/30
7/7 - 8s - loss: 0.6547 - f1: 0.6830 - precision: 0.6830 - recall: 0.6830 - accuracy: 0.6396 - val_loss: 0.6423 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 5/30
7/7 - 8s - loss: 0.6556 - f1: 0.6589 - precision: 0.6589 - recall: 0.6589 - accuracy: 0.6396 - val_loss: 0.6419 - val_f1: 0.6736 - val_precision: 0.6736 - va

In [ ]:
from tensorflow.keras.utils import to_categorical
ycb_x1= tf.keras.applications.vgg16.preprocess_input(np.array(ycb_x))
ycb_xtrain1, ycb_xtest1, ycb_ytrain, ycb_ytest= train_test_split(ycb_x1, ycb_y, random_state=1, train_size=0.75)
ycb_ytrain=to_categorical(ycb_ytrain,2)

from tensorflow.keras.callbacks import EarlyStopping

model.fit(ycb_xtrain1, ycb_ytrain, epochs=30, validation_split=0.2, verbose=2, callbacks=[custom_early_stopping])
pred=np.argmax(model.predict(np.array(ycb_xtest1)),axis=1)
vgg_ycb_cm, vgg_ycb_acc, vgg_ycb_f1, vgg_ycb_sen= score(ycb_ytest, pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
7/7 - 9s - loss: 0.6534 - f1: 0.6589 - precision: 0.6589 - recall: 0.6589 - accuracy: 0.6396 - val_loss: 0.6416 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 2/30
7/7 - 9s - loss: 0.6544 - f1: 0.6589 - precision: 0.6589 - recall: 0.6589 - accuracy: 0.6396 - val_loss: 0.6414 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 3/30
7/7 - 9s - loss: 0.6537 - f1: 0.6348 - precision: 0.6348 - recall: 0.6348 - accuracy: 0.6396 - val_loss: 0.6413 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 4/30
7/7 - 8s - loss: 0.6533 - f1: 0.6107 - precision: 0.6107 - recall: 0.6107 - accuracy: 0.6396 - val_loss: 0.6414 - val_f1: 0.6736 - val_precision: 0.6736 - val_recall: 0.6736 - val_accuracy: 0.6600
Epoch 5/30
7/7 - 9s - loss: 0.6540 - f1: 0.6107 - precision: 0.6107 - recall: 0.6107 - accuracy: 0.6396 - val_loss: 0.6416 - val_f1: 0.6736 - val_precision: 0.6736 - va

In [ ]:
rgb_y1= to_categorical(rgb_y,2)
kfold(inputs= np.array(rgb_x1), targets= np.array(gray_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=model)

------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/10
8/8 [==============================] - 9s 1s/step - loss: 0.6461 - f1: 0.6542 - precision: 0.6542 - recall: 0.6542 - accuracy: 0.6559
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.6431 - f1: 0.6527 - precision: 0.6527 - recall: 0.6527 - accuracy: 0.6559
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.6438 - f1: 0.6542 - precision: 0.6542 - recall: 0.6542 - accuracy: 0.6559
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.6436 - f1: 0.6557 - precision: 0.6557 - recall: 0.6557 - accuracy: 0.6559
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.6432 - f1: 0.6573 - precision: 0.6573 - recall: 0.6573 - accuracy: 0.6559
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.6444 - f1: 0.6512 - precision: 0.6512 - recall: 0.6512 - accuracy: 0.6559
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.6449 - f1: 0.6588 - precision: 0.6588 - recall: 0.6588 - ac

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
hsv_y1= to_categorical(hsv_y,2)
kfold(inputs= np.array(hsv_x1), targets= np.array(gray_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=model)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 9s 1s/step - loss: 0.6229 - f1: 0.6885 - precision: 0.6885 - recall: 0.6885 - accuracy: 0.6883
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.6219 - f1: 0.6916 - precision: 0.6916 - recall: 0.6916 - accuracy: 0.6883
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.6222 - f1: 0.6824 - precision: 0.6824 - recall: 0.6824 - accuracy: 0.6883
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.6219 - f1: 0.6870 - precision: 0.6870 - recall: 0.6870 - accuracy: 0.6883
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.6203 - f1: 0.6931 - precision: 0.6931 - recall: 0.6931 - accuracy: 0.6883
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.6210 - f1: 0.6870 - precision: 0.6870 - recall: 0.6870 - accuracy: 0.6883
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.6221 - f1: 0.6885 - precision: 0.6885 - recall: 0.6885 - accuracy: 0.6

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
lab_y1= to_categorical(lab_y,2)
kfold(inputs= np.array(lab_x1), targets= np.array(gray_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=model)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 9s 1s/step - loss: 0.6565 - f1: 0.6362 - precision: 0.6362 - recall: 0.6362 - accuracy: 0.6356
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.6575 - f1: 0.6316 - precision: 0.6316 - recall: 0.6316 - accuracy: 0.6356
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.6558 - f1: 0.6377 - precision: 0.6377 - recall: 0.6377 - accuracy: 0.6356
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.6557 - f1: 0.6362 - precision: 0.6362 - recall: 0.6362 - accuracy: 0.6356
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.6566 - f1: 0.6316 - precision: 0.6316 - recall: 0.6316 - accuracy: 0.6356
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.6557 - f1: 0.6362 - precision: 0.6362 - recall: 0.6362 - accuracy: 0.6356
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.6565 - f1: 0.6393 - precision: 0.6393 - recall: 0.6393 - accuracy: 0.6

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
ycb_y1= to_categorical(ycb_y,2)
kfold(inputs= np.array(ycb_x1), targets= np.array(gray_y1), batch_size=32, no_epochs=10, num_folds=4, verbosity=1, ori_model=model)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 9s 1s/step - loss: 0.6268 - f1: 0.6877 - precision: 0.6877 - recall: 0.6877 - accuracy: 0.6842
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.6256 - f1: 0.6861 - precision: 0.6861 - recall: 0.6861 - accuracy: 0.6842
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.6266 - f1: 0.6877 - precision: 0.6877 - recall: 0.6877 - accuracy: 0.6842
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.6236 - f1: 0.6831 - precision: 0.6831 - recall: 0.6831 - accuracy: 0.6842
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.6244 - f1: 0.6831 - precision: 0.6831 - recall: 0.6831 - accuracy: 0.6842
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.6241 - f1: 0.6892 - precision: 0.6892 - recall: 0.6892 - accuracy: 0.6842
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.6234 - f1: 0.6892 - precision: 0.6892 - recall: 0.6892 - accuracy: 0.6

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# resnet  

In [ ]:
import os, sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import keras
import pickle
import PIL.Image
import random
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
import time
from tensorflow.keras.utils import to_categorical
def res_kfold(inputs, targets, batch_size, no_epochs, num_folds, verbosity):
    start_time= time.time()
    print(start_time)
    targets= to_categorical(targets)
    kfold = KFold(n_splits=num_folds, shuffle=True)
    acc_per_fold =[]
    loss_per_fold =[]
    f1_per_fold =[]
    sen_per_fold =[]
    spec_per_fold =[]
    pre_per_fold=[]
    fold_no = 1
    for train, test in kfold.split(inputs, targets):
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')


      # Fit data to model
        history = resnet_model.fit(inputs[train], targets[train],
                  batch_size=batch_size,
                  epochs=no_epochs)

              # Fit data to model

        scores = resnet_model.evaluate(inputs[test], targets[test], verbose=0)
        print(scores)
        print(f'Score for fold {fold_no}: {resnet_model.metrics_names} of {scores}')
        acc_per_fold.append(scores[4])
        loss_per_fold.append(scores[0])
        f1_per_fold.append(scores[1])
        sen_per_fold.append(scores[2])
        pre_per_fold.append(scores[3])

      # Increase fold number
        fold_no = fold_no + 1
        if fold_no>4:
            break
    print(acc_per_fold)
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {np.around(loss_per_fold[i], decimals=4)} - Accuracy: {np.around(acc_per_fold[i], decimals=4)}%')
        print(f'> Fold {i+1} - f1: {np.around(f1_per_fold[i], decimals=4)} - sensitivity: {np.around(sen_per_fold[i],decimals=4)}- precision:{np.around(pre_per_fold[i],decimals=4)}')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> accuracy: {np.around(np.mean(acc_per_fold), decimals=4)} (+- {np.around(np.std(acc_per_fold), decimals=4)})')
    print(f'> precision: {np.around(np.mean(pre_per_fold),decimals=4)} (+- {np.around(np.std(pre_per_fold), decimals=4)})')
    print(f'> f1: {np.around(np.mean(f1_per_fold),decimals=4)} (+- {np.around(np.std(f1_per_fold),decimals=4)})')
    print(f'> sensitivity: {np.around(np.mean(sen_per_fold),decimals=4)} (+- {np.around(np.std(sen_per_fold),decimals=4)})')
    print(f'> specificity: {np.around(np.mean(spec_per_fold),decimals=4)} (+- {np.around(np.std(spec_per_fold),decimals=4)})')
    print('------------------------------------------------------------------------')
    end_time=time.time()
    print('time', end_time-start_time)

In [ ]:
def resnet_train_test_split(x,y):
    xtrain, xtest, ytrain, ytest= train_test_split(x, y, random_state=1, train_size=0.75)
    if len(np.shape(xtrain))==4:
        xtrain=np.array(xtrain)/255
        xtest=np.array(xtest)/255
    else:
        np.array(xtrain).reshape(-1, 256,256,1)/255
    ytrain=np.array(ytrain)
    ytrain= np.asarray(ytrain).reshape((-1,1))
    ytrain=to_categorical(ytrain)
    return xtrain, ytrain, xtest, ytest

In [ ]:
gray_x1= np.expand_dims(gray_x, axis=3)
rgb_xtrain2, rgb_ytrain, rgb_xtest2, rgb_ytest=resnet_train_test_split(rgb_x,rgb_y)
hsv_xtrain2, hsv_ytrain, hsv_xtest2, hsv_ytest=resnet_train_test_split(hsv_x,hsv_y)
gray_xtrain2, gray_ytrain, gray_xtest2, gray_ytest=resnet_train_test_split(gray_x1,gray_y)
lab_xtrain2, lab_ytrain, lab_xtest2, lab_ytest=resnet_train_test_split(lab_x,lab_y)
ycb_xtrain2, ycb_ytrain, ycb_xtest2, ycb_ytest=resnet_train_test_split(ycb_x,ycb_y)

In [ ]:
shape, classes = (224, 224, 3), 2
resnet_model = ResNet50(input_shape = shape, weights=None, classes=classes)
resnet_model.compile(optimizer="adam", loss='binary_crossentropy', metrics=[f1, recall, precision, "accuracy"])

In [ ]:
tf.config.run_functions_eagerly(True)
resnet_model.fit(tf.stack(rgb_xtrain2), tf.stack(rgb_ytrain), epochs=40, batch_size=6, callbacks=[custom_early_stopping])
y_pred=np.argmax(resnet_model.predict(rgb_xtest2),axis=1)
res_rgb_cm, res_rgb_acc, res_rgb_f1, res_rgb_sen= score(y_pred, rgb_ytest)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/40
42/42 [==============================] - 12s 268ms/step - loss: 1.2115 - f1: 0.5754 - recall: 0.5754 - precision: 0.5754 - accuracy: 0.5668
Epoch 2/40
42/42 [==============================] - 11s 268ms/step - loss: 0.8560 - f1: 0.6111 - recall: 0.6111 - precision: 0.6111 - accuracy: 0.6235
Epoch 3/40
42/42 [==============================] - 11s 261ms/step - loss: 0.6648 - f1: 0.6627 - recall: 0.6627 - precision: 0.6627 - accuracy: 0.6559
Epoch 4/40
42/42 [==============================] - 11s 261ms/step - loss: 0.6923 - f1: 0.6270 - recall: 0.6270 - precision: 0.6270 - accuracy: 0.6194
Epoch 5/40
42/42 [==============================] - 11s 261ms/step - loss: 0.6261 - f1: 0.6825 - recall: 0.6825 - precision: 0.6825 - accuracy: 0.6761
Epoch 6/40
42/42 [==============================] - 11s 261ms/step - loss: 0.6381 - f1: 0.6746 - recall: 0.6746 - precision: 0.6746 - accuracy: 0.6680
Epoch 7/40
42/42 [==============================] - 11s 265ms/step - loss: 0.6568 - f1: 0.6706

In [ ]:
resnet_model.fit(tf.stack(hsv_xtrain2), tf.stack(hsv_ytrain), epochs=10, batch_size=6, callbacks=[custom_early_stopping])
y_pred=np.argmax(resnet_model.predict(hsv_xtest2),axis=1)
res_hsv_cm, res_hsv_acc, res_hsv_f1, res_hsv_sen= score(y_pred, hsv_ytest)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


42/42 [==============================] - 11s 262ms/step - loss: 0.6993 - f1: 0.6865 - recall: 0.6865 - precision: 0.6865 - accuracy: 0.6802
Epoch 2/10
42/42 [==============================] - 11s 267ms/step - loss: 0.4621 - f1: 0.7897 - recall: 0.7897 - precision: 0.7897 - accuracy: 0.7854
Epoch 3/10
42/42 [==============================] - 11s 262ms/step - loss: 0.4412 - f1: 0.7897 - recall: 0.7897 - precision: 0.7897 - accuracy: 0.7854
Epoch 4/10
42/42 [==============================] - 11s 262ms/step - loss: 0.3270 - f1: 0.8492 - recall: 0.8492 - precision: 0.8492 - accuracy: 0.8664
Epoch 5/10
42/42 [==============================] - 11s 265ms/step - loss: 0.3123 - f1: 0.8770 - recall: 0.8770 - precision: 0.8770 - accuracy: 0.8745
Epoch 6/10
42/42 [==============================] - 11s 266ms/step - loss: 0.1751 - f1: 0.9286 - recall: 0.9286 - precision: 0.9286 - accuracy: 0.9271
Epoch 7/10
42/42 [==============================] - 11s 264ms/step - loss: 0.2426 - f1: 0.9087 - recall: 

In [ ]:
resnet_model.fit(tf.stack(lab_xtrain2), tf.stack(lab_ytrain), epochs=30, batch_size=6, callbacks=[custom_early_stopping])
y_pred=np.argmax(resnet_model.predict(lab_xtest2),axis=1)
res_lab_cm, res_lab_acc, res_lab_f1, res_lab_sen= score(y_pred, lab_ytest)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
42/42 [==============================] - 11s 268ms/step - loss: 0.6819 - f1: 0.7103 - recall: 0.7103 - precision: 0.7103 - accuracy: 0.7045
Epoch 2/30
42/42 [==============================] - 11s 263ms/step - loss: 0.4541 - f1: 0.8294 - recall: 0.8294 - precision: 0.8294 - accuracy: 0.8259
Epoch 3/30
42/42 [==============================] - 11s 263ms/step - loss: 0.3772 - f1: 0.8135 - recall: 0.8135 - precision: 0.8135 - accuracy: 0.8300
Epoch 4/30
42/42 [==============================] - 11s 263ms/step - loss: 0.2524 - f1: 0.9087 - recall: 0.9087 - precision: 0.9087 - accuracy: 0.9069
Epoch 5/30
42/42 [==============================] - 11s 261ms/step - loss: 0.3268 - f1: 0.8452 - recall: 0.8452 - precision: 0.8452 - accuracy: 0.8623
Epoch 6/30
42/42 [==============================] - 11s 263ms/step - loss: 0.2303 - f1: 0.8968 - recall: 0.8968 - precision: 0.8968 - accuracy: 0.9150
Epoch 7/30
42/42 [==============================] - 11s 268ms/step - loss: 0.2028 - f1: 0.9048

In [ ]:
resnet_model.fit(tf.stack(ycb_xtrain2), tf.stack(ycb_ytrain), epochs=50, batch_size=6, callbacks=[custom_early_stopping])
y_pred=np.argmax(resnet_model.predict(ycb_xtest2),axis=1)
res_ycb_cm, res_ycb_acc, res_ycb_f1, res_ycb_sen= score(y_pred, ycb_ytest)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


42/42 [==============================] - 11s 267ms/step - loss: 0.2426 - f1: 0.9246 - recall: 0.9246 - precision: 0.9246 - accuracy: 0.9231
Epoch 2/50
42/42 [==============================] - 11s 263ms/step - loss: 0.0769 - f1: 0.9762 - recall: 0.9762 - precision: 0.9762 - accuracy: 0.9757
Epoch 3/50
42/42 [==============================] - 11s 262ms/step - loss: 0.0469 - f1: 0.9841 - recall: 0.9841 - precision: 0.9841 - accuracy: 0.9838
Epoch 4/50
42/42 [==============================] - 11s 263ms/step - loss: 0.0574 - f1: 0.9762 - recall: 0.9762 - precision: 0.9762 - accuracy: 0.9757
Epoch 5/50
42/42 [==============================] - 11s 262ms/step - loss: 0.0395 - f1: 0.9841 - recall: 0.9841 - precision: 0.9841 - accuracy: 0.9838
Epoch 6/50
42/42 [==============================] - 11s 266ms/step - loss: 0.0418 - f1: 0.9802 - recall: 0.9802 - precision: 0.9802 - accuracy: 0.9798
Epoch 7/50
42/42 [==============================] - 11s 263ms/step - loss: 0.0374 - f1: 0.9841 - recall: 

In [ ]:
shape, classes = (224, 224, 1), 2
resnet_model1 = ResNet50(input_shape = shape, weights=None, classes=classes)
resnet_model1.compile(optimizer="adam", loss='binary_crossentropy', metrics=[f1, recall, precision,"accuracy"])

In [ ]:
resnet_model1.fit(tf.stack(gray_xtrain2), tf.stack(gray_ytrain), epochs=50, batch_size=6, callbacks=[custom_early_stopping])
y_pred=np.argmax(resnet_model1.predict(gray_xtest2),axis=1)
res_gray_cm, res_gray_acc, res_gray_f1, res_gray_sen= score(y_pred, gray_ytest)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/50
42/42 [==============================] - 11s 261ms/step - loss: 1.1526 - f1: 0.5952 - recall: 0.5952 - precision: 0.5952 - accuracy: 0.6073
Epoch 2/50
42/42 [==============================] - 11s 259ms/step - loss: 0.7113 - f1: 0.5952 - recall: 0.5952 - precision: 0.5952 - accuracy: 0.6073
Epoch 3/50
42/42 [==============================] - 11s 260ms/step - loss: 0.8742 - f1: 0.5119 - recall: 0.5119 - precision: 0.5119 - accuracy: 0.5223
Epoch 4/50
42/42 [==============================] - 11s 263ms/step - loss: 0.7257 - f1: 0.6270 - recall: 0.6270 - precision: 0.6270 - accuracy: 0.6194
Epoch 5/50
42/42 [==============================] - 11s 257ms/step - loss: 0.6076 - f1: 0.6984 - recall: 0.6984 - precision: 0.6984 - accuracy: 0.7126
Epoch 6/50
42/42 [==============================] - 11s 259ms/step - loss: 0.6682 - f1: 0.6190 - recall: 0.6190 - precision: 0.6190 - accuracy: 0.6113
Epoch 7/50
42/42 [==============================] - 11s 256ms/step - loss: 0.6318 - f1: 0.6905

In [ ]:
res_kfold(inputs= np.array(gray_xsd), targets= np.array(gray_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631162877.0834463
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


ValueError: ignored

In [ ]:
shape, classes = (224, 224, 3), 2
resnet_model1 = ResNet50(input_shape = shape, weights=None, classes=classes)
resnet_model1.compile(optimizer="adam", loss='binary_crossentropy', metrics=[f1, recall, precision,"accuracy"])

In [ ]:
res_kfold(inputs= np.array(rgb_x), targets= np.array(rgb_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631162903.2319405
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 8s 919ms/step - loss: 1.2041 - f1: 0.7128 - recall: 0.7128 - precision: 0.7128 - accuracy: 0.7166
Epoch 2/15
8/8 [==============================] - 6s 781ms/step - loss: 0.4784 - f1: 0.8142 - recall: 0.8142 - precision: 0.8142 - accuracy: 0.8138
Epoch 3/15
8/8 [==============================] - 6s 778ms/step - loss: 0.2809 - f1: 0.8915 - recall: 0.8915 - precision: 0.8915 - accuracy: 0.8907
Epoch 4/15
8/8 [==============================] - 6s 780ms/step - loss: 0.1641 - f1: 0.9531 - recall: 0.9531 - precision: 0.9531 - accuracy: 0.9514
Epoch 5/15
8/8 [==============================] - 6s 778ms/step - loss: 0.1226 - f1: 0.9883 - recall: 0.9883 - precision: 0.9883 - accuracy: 0.9879
Epoch 6/15
8/8 [==============================] - 6s 777ms/step - loss: 0.0718 - f1: 0.9922 - recall: 0.9922 - precision: 0.9922 - accuracy: 0.9919
Epoch 7/15
8/8 [==============================] - 6s 777ms/step - loss: 0.0443 - f1: 1.0000 - recall: 1.0000 - precision: 1

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
res_kfold(inputs= np.array(hsv_x), targets= np.array(hsv_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631163428.6941748
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 6s 779ms/step - loss: 1.4859 - f1: 0.6814 - recall: 0.6814 - precision: 0.6814 - accuracy: 0.6761
Epoch 2/15
8/8 [==============================] - 6s 776ms/step - loss: 0.3439 - f1: 0.8619 - recall: 0.8619 - precision: 0.8619 - accuracy: 0.8664
Epoch 3/15
8/8 [==============================] - 6s 780ms/step - loss: 0.1827 - f1: 0.9368 - recall: 0.9368 - precision: 0.9368 - accuracy: 0.9393
Epoch 4/15
8/8 [==============================] - 6s 778ms/step - loss: 0.1167 - f1: 0.9868 - recall: 0.9868 - precision: 0.9868 - accuracy: 0.9879
Epoch 5/15
8/8 [==============================] - 6s 776ms/step - loss: 0.0646 - f1: 0.9922 - recall: 0.9922 - precision: 0.9922 - accuracy: 0.9919
Epoch 6/15
8/8 [==============================] - 6s 777ms/step - loss: 0.0438 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 7/15
8/8 [==============================] - 6s 776ms/step - loss: 0.0261 - f1: 1.0000 - recall: 1.0000 - precision: 1

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
res_kfold(inputs= np.array(lab_x), targets= np.array(lab_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631163952.6726549
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 6s 777ms/step - loss: 0.4772 - f1: 0.8563 - recall: 0.8563 - precision: 0.8563 - accuracy: 0.8543
Epoch 2/15
8/8 [==============================] - 6s 778ms/step - loss: 0.1715 - f1: 0.9251 - recall: 0.9251 - precision: 0.9251 - accuracy: 0.9271
Epoch 3/15
8/8 [==============================] - 6s 781ms/step - loss: 0.0669 - f1: 0.9844 - recall: 0.9844 - precision: 0.9844 - accuracy: 0.9838
Epoch 4/15
8/8 [==============================] - 6s 778ms/step - loss: 0.0577 - f1: 0.9789 - recall: 0.9789 - precision: 0.9789 - accuracy: 0.9798
Epoch 5/15
8/8 [==============================] - 6s 778ms/step - loss: 0.0139 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 6/15
8/8 [==============================] - 6s 779ms/step - loss: 0.0099 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 7/15
8/8 [==============================] - 6s 776ms/step - loss: 0.0082 - f1: 1.0000 - recall: 1.0000 - precision: 1

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
res_kfold(inputs= np.array(ycb_x), targets= np.array(ycb_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631164379.622365
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 6s 774ms/step - loss: 0.0439 - f1: 0.9907 - recall: 0.9907 - precision: 0.9907 - accuracy: 0.9919
Epoch 2/15
8/8 [==============================] - 6s 776ms/step - loss: 0.0096 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 3/15
8/8 [==============================] - 6s 774ms/step - loss: 0.0081 - f1: 1.0000 - recall: 1.0000 - precision: 1.0000 - accuracy: 1.0000
Epoch 4/15
8/8 [==============================] - 6s 773ms/step - loss: 0.0164 - f1: 0.9961 - recall: 0.9961 - precision: 0.9961 - accuracy: 0.9960
Epoch 5/15
8/8 [==============================] - 6s 775ms/step - loss: 0.0167 - f1: 0.9907 - recall: 0.9907 - precision: 0.9907 - accuracy: 0.9919
Epoch 6/15
8/8 [==============================] - 6s 774ms/step - loss: 0.0068 - f1: 0.9961 - recall: 0.9961 - precision: 0.9961 - accuracy: 0.9960
Epoch 7/15
8/8 [==============================] - 6s 774ms/step - loss: 0.0029 - f1: 1.0000 - recall: 1.0000 - precision: 1

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# DenseNet

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
#使用tf.keras.applications中的DenseNet121网络，并且使用官方的预训练模型
covn_base = tf.keras.applications.DenseNet121(weights='imagenet',include_top=False,input_shape=(224,224,3))
covn_base.trainable = True

#冻结前面的层，训练最后5层
for layers in covn_base.layers[:-5]:
    layers.trainable = False

#构建模型
model = tf.keras.Sequential()
model.add(covn_base)
model.add(tf.keras.layers.GlobalAveragePooling2D())  #加入全局平均池化层
model.add(tf.keras.layers.Dense(512,activation='relu'))  #添加全连接层
model.add(tf.keras.layers.Dropout(rate=0.5))  #添加Dropout层，防止过拟合

model.add(tf.keras.layers.Dense(3,activation='softmax'))  #添加输出层(2分类)
model.summary()   #打印每层参数信息

#编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), #使用adam优化器，学习率为0.0001
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), #交叉熵损失函数
              metrics=[f1, recall, precision,'accuracy']) #评价函数

29097984/29084464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
Total params: 7,563,843
Trainable params: 565,251
Non-trainable params: 6,998,592
_________________________________________________________________


In [ ]:
import time
from tensorflow.keras.utils import to_categorical
def dense_kfold(inputs, targets, batch_size, no_epochs, num_folds, verbosity):
    start_time= time.time()
    print(start_time)
    targets= to_categorical(targets)
    kfold = KFold(n_splits=num_folds, shuffle=True)
    acc_per_fold =[]
    loss_per_fold =[]
    f1_per_fold =[]
    sen_per_fold =[]
    spec_per_fold =[]
    pre_per_fold=[]
    fold_no = 1
    for train, test in kfold.split(inputs, targets):
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')


      # Fit data to model
        history = model.fit(inputs[train], targets[train],
                  batch_size=batch_size,
                  epochs=no_epochs)

              # Fit data to model

        scores = model.evaluate(inputs[test], targets[test], verbose=0)
        print(scores)
        print(f'Score for fold {fold_no}: {model.metrics_names} of {scores}')
        acc_per_fold.append(scores[4])
        loss_per_fold.append(scores[0])
        f1_per_fold.append(scores[1])
        sen_per_fold.append(scores[2])
        pre_per_fold.append(scores[3])

      # Increase fold number
        fold_no = fold_no + 1
        if fold_no>4:
            break
    print(acc_per_fold)
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {np.around(loss_per_fold[i], decimals=4)} - Accuracy: {np.around(acc_per_fold[i], decimals=4)}%')
        print(f'> Fold {i+1} - f1: {np.around(f1_per_fold[i], decimals=4)} - sensitivity: {np.around(sen_per_fold[i],decimals=4)}- precision:{np.around(pre_per_fold[i],decimals=4)}')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> accuracy: {np.around(np.mean(acc_per_fold), decimals=4)} (+- {np.around(np.std(acc_per_fold), decimals=4)})')
    print(f'> precision: {np.around(np.mean(pre_per_fold),decimals=4)} (+- {np.around(np.std(pre_per_fold), decimals=4)})')
    print(f'> f1: {np.around(np.mean(f1_per_fold),decimals=4)} (+- {np.around(np.std(f1_per_fold),decimals=4)})')
    print(f'> sensitivity: {np.around(np.mean(sen_per_fold),decimals=4)} (+- {np.around(np.std(sen_per_fold),decimals=4)})')
    print(f'> specificity: {np.around(np.mean(spec_per_fold),decimals=4)} (+- {np.around(np.std(spec_per_fold),decimals=4)})')
    print('------------------------------------------------------------------------')
    end_time=time.time()
    print('time', end_time-start_time)

In [ ]:
model.fit(tf.stack(rgb_xtrain), tf.stack(rgb_ytrain), epochs=30, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(model.predict(np.array(rgb_xtest)),axis=1)
dense_rgb_cm, dense_rgb_acc, dense_rgb_f1, dense_rgb_sen= score(rgb_ytest, prediction)

Epoch 1/30
7/7 [==============================] - 41s 442ms/step - loss: 1.1834 - f1: 0.3579 - recall: 0.3240 - precision: 0.4102 - accuracy: 0.4494
Epoch 2/30
7/7 [==============================] - 2s 219ms/step - loss: 0.9324 - f1: 0.5055 - recall: 0.4408 - precision: 0.5940 - accuracy: 0.5385
Epoch 3/30
7/7 [==============================] - 2s 219ms/step - loss: 0.8080 - f1: 0.6095 - recall: 0.5566 - precision: 0.6742 - accuracy: 0.6275
Epoch 4/30
7/7 [==============================] - 2s 221ms/step - loss: 0.7240 - f1: 0.6386 - recall: 0.5944 - precision: 0.6960 - accuracy: 0.6640
Epoch 5/30
7/7 [==============================] - 2s 219ms/step - loss: 0.6020 - f1: 0.7122 - recall: 0.6755 - precision: 0.7543 - accuracy: 0.7368
Epoch 6/30
7/7 [==============================] - 2s 223ms/step - loss: 0.6681 - f1: 0.7081 - recall: 0.6781 - precision: 0.7423 - accuracy: 0.7287
Epoch 7/30
7/7 [==============================] - 2s 222ms/step - loss: 0.5655 - f1: 0.7364 - recall: 0.6959 - 

In [ ]:
model.fit(tf.stack(hsv_xtrain), tf.stack(hsv_ytrain), epochs=30, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(model.predict(np.array(hsv_xtest)),axis=1)
dense_hsv_cm, dense_hsv_acc, dense_hsv_f1, dense_hsv_sen= score(hsv_ytest, prediction)

Epoch 1/30
7/7 [==============================] - 2s 230ms/step - loss: 1.4113 - f1: 0.3855 - recall: 0.3480 - precision: 0.4339 - accuracy: 0.4332
Epoch 2/30
7/7 [==============================] - 2s 223ms/step - loss: 1.0515 - f1: 0.5188 - recall: 0.4694 - precision: 0.5821 - accuracy: 0.5466
Epoch 3/30
7/7 [==============================] - 2s 221ms/step - loss: 0.8125 - f1: 0.6083 - recall: 0.5413 - precision: 0.6964 - accuracy: 0.6073
Epoch 4/30
7/7 [==============================] - 2s 219ms/step - loss: 0.6787 - f1: 0.7101 - recall: 0.6653 - precision: 0.7658 - accuracy: 0.6964
Epoch 5/30
7/7 [==============================] - 2s 221ms/step - loss: 0.6651 - f1: 0.7487 - recall: 0.7199 - precision: 0.7809 - accuracy: 0.7611
Epoch 6/30
7/7 [==============================] - 2s 221ms/step - loss: 0.5931 - f1: 0.7472 - recall: 0.7148 - precision: 0.7837 - accuracy: 0.8057
Epoch 7/30
7/7 [==============================] - 2s 221ms/step - loss: 0.5375 - f1: 0.7832 - recall: 0.7449 - p

In [ ]:
model.fit(tf.stack(lab_xtrain), tf.stack(lab_ytrain), epochs=30, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(model.predict(np.array(lab_xtest)),axis=1)
dense_lab_cm, dense_lab_acc, dense_lab_f1, dense_lab_sen= score(lab_ytest, prediction)

Epoch 1/30
7/7 [==============================] - 2s 230ms/step - loss: 1.3891 - f1: 0.2670 - recall: 0.2230 - precision: 0.3334 - accuracy: 0.3441
Epoch 2/30
7/7 [==============================] - 2s 220ms/step - loss: 1.1971 - f1: 0.3463 - recall: 0.2990 - precision: 0.4128 - accuracy: 0.4413
Epoch 3/30
7/7 [==============================] - 2s 218ms/step - loss: 1.0651 - f1: 0.3713 - recall: 0.3204 - precision: 0.4493 - accuracy: 0.4858
Epoch 4/30
7/7 [==============================] - 2s 218ms/step - loss: 0.9559 - f1: 0.4886 - recall: 0.4276 - precision: 0.5719 - accuracy: 0.5709
Epoch 5/30
7/7 [==============================] - 2s 218ms/step - loss: 0.8800 - f1: 0.5563 - recall: 0.4888 - precision: 0.6494 - accuracy: 0.5789
Epoch 6/30
7/7 [==============================] - 2s 220ms/step - loss: 0.8251 - f1: 0.5894 - recall: 0.5041 - precision: 0.7151 - accuracy: 0.6437
Epoch 7/30
7/7 [==============================] - 2s 218ms/step - loss: 0.7944 - f1: 0.6166 - recall: 0.5556 - p

In [ ]:
tf.config.run_functions_eagerly(True)
model.fit(tf.stack(ycb_xtrain), tf.stack(ycb_ytrain), epochs=30, batch_size=40,callbacks=[custom_early_stopping])
prediction=np.argmax(model.predict(np.array(ycb_xtest)),axis=1)
dense_ycb_cm, dense_ycb_acc, dense_ycb_f1, dense_ycb_sen= score(ycb_ytest, prediction)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/30
7/7 [==============================] - 6s 826ms/step - loss: 0.8839 - f1: 0.5836 - recall: 0.5291 - precision: 0.6525 - accuracy: 0.6356
Epoch 2/30
7/7 [==============================] - 6s 798ms/step - loss: 0.8194 - f1: 0.6080 - recall: 0.5398 - precision: 0.7026 - accuracy: 0.6437
Epoch 3/30
7/7 [==============================] - 6s 792ms/step - loss: 0.7858 - f1: 0.6357 - recall: 0.5781 - precision: 0.7108 - accuracy: 0.6721
Epoch 4/30
7/7 [==============================] - 6s 796ms/step - loss: 0.8105 - f1: 0.6405 - recall: 0.5903 - precision: 0.7033 - accuracy: 0.6194
Epoch 5/30
7/7 [==============================] - 6s 802ms/step - loss: 0.7855 - f1: 0.6491 - recall: 0.5974 - precision: 0.7194 - accuracy: 0.6316
Epoch 6/30
7/7 [==============================] - 6s 794ms/step - loss: 0.7682 - f1: 0.6499 - recall: 0.5923 - precision: 0.7214 - accuracy: 0.6964
Epoch 7/30
7/7 [==============================] - 6s 800ms/step - loss: 0.7203 - f1: 0.6858 - recall: 0.6413 - p

In [ ]:
a=np.array([gray_x,gray_x,gray_x])
gray_x2= a.transpose(1,2,3,0)
gray_xtrain3, gray_ytrain, gray_xtest2, gray_ytest=resnet_train_test_split(gray_x2,gray_y)

In [ ]:
tf.config.run_functions_eagerly(True)
model.fit(tf.stack(gray_xtrain3), tf.stack(gray_ytrain), epochs=20, batch_size=10,callbacks=[custom_early_stopping])
prediction=np.argmax(model.predict(np.array(gray_xtest2)),axis=1)
dense_gray_cm, dense_gray_acc, dense_gray_f1, dense_gray_sen= score(gray_ytest, prediction)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/20
25/25 [==============================] - 11s 412ms/step - loss: 0.1867 - f1: 0.9594 - recall: 0.9520 - precision: 0.9676 - accuracy: 0.9636
Epoch 2/20
25/25 [==============================] - 10s 415ms/step - loss: 0.1550 - f1: 0.9630 - recall: 0.9566 - precision: 0.9706 - accuracy: 0.9757
Epoch 3/20
25/25 [==============================] - 10s 408ms/step - loss: 0.1813 - f1: 0.9537 - recall: 0.9463 - precision: 0.9618 - accuracy: 0.9636
Epoch 4/20
25/25 [==============================] - 10s 416ms/step - loss: 0.1451 - f1: 0.9836 - recall: 0.9760 - precision: 0.9920 - accuracy: 0.9919
Epoch 5/20
25/25 [==============================] - 10s 416ms/step - loss: 0.1862 - f1: 0.9307 - recall: 0.9183 - precision: 0.9445 - accuracy: 0.9352
Epoch 6/20
25/25 [==============================] - 10s 410ms/step - loss: 0.1421 - f1: 0.9676 - recall: 0.9640 - precision: 0.9716 - accuracy: 0.9676
Epoch 7/20
25/25 [==============================] - 10s 418ms/step - loss: 0.1415 - f1: 0.9724

In [ ]:
dense_kfold(inputs= np.array(gray_x), targets= np.array(gray_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631165875.0120013
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


ValueError: ignored

In [ ]:
dense_kfold(inputs= np.array(rgb_x), targets= np.array(rgb_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631166471.8790894
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 6s 733ms/step - loss: 0.8475 - f1: 0.5948 - recall: 0.5948 - precision: 0.5948 - accuracy: 0.5911
Epoch 2/15
8/8 [==============================] - 6s 740ms/step - loss: 0.6674 - f1: 0.6715 - recall: 0.6715 - precision: 0.6715 - accuracy: 0.6802
Epoch 3/15
8/8 [==============================] - 6s 737ms/step - loss: 0.7107 - f1: 0.6442 - recall: 0.6442 - precision: 0.6442 - accuracy: 0.6518
Epoch 4/15
8/8 [==============================] - 6s 733ms/step - loss: 0.6432 - f1: 0.6963 - recall: 0.6963 - precision: 0.6963 - accuracy: 0.6964
Epoch 5/15
8/8 [==============================] - 6s 737ms/step - loss: 0.6029 - f1: 0.7026 - recall: 0.7026 - precision: 0.7026 - accuracy: 0.7045
Epoch 6/15
8/8 [==============================] - 6s 735ms/step - loss: 0.5252 - f1: 0.7252 - recall: 0.7252 - precision: 0.7252 - accuracy: 0.7247
Epoch 7/15
8/8 [==============================] - 6s 733ms/step - loss: 0.5236 - f1: 0.7493 - recall: 0.7493 - precision: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
dense_kfold(inputs= np.array(hsv_x), targets= np.array(hsv_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631166885.0698252
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 6s 735ms/step - loss: 0.8208 - f1: 0.6433 - recall: 0.6433 - precision: 0.6433 - accuracy: 0.6478
Epoch 2/15
8/8 [==============================] - 6s 731ms/step - loss: 0.7581 - f1: 0.5917 - recall: 0.5917 - precision: 0.5917 - accuracy: 0.5911
Epoch 3/15
8/8 [==============================] - 6s 734ms/step - loss: 0.7180 - f1: 0.6518 - recall: 0.6518 - precision: 0.6518 - accuracy: 0.6518
Epoch 4/15
8/8 [==============================] - 6s 732ms/step - loss: 0.5911 - f1: 0.7057 - recall: 0.7057 - precision: 0.7057 - accuracy: 0.7045
Epoch 5/15
8/8 [==============================] - 6s 739ms/step - loss: 0.5446 - f1: 0.7376 - recall: 0.7376 - precision: 0.7376 - accuracy: 0.7328
Epoch 6/15
8/8 [==============================] - 6s 733ms/step - loss: 0.5245 - f1: 0.7128 - recall: 0.7128 - precision: 0.7128 - accuracy: 0.7166
Epoch 7/15
8/8 [==============================] - 6s 734ms/step - loss: 0.5013 - f1: 0.7792 - recall: 0.7792 - precision: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
dense_kfold(inputs= np.array(lab_x), targets= np.array(lab_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631167298.7835166
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 6s 757ms/step - loss: 0.7775 - f1: 0.6456 - recall: 0.6456 - precision: 0.6456 - accuracy: 0.6437
Epoch 2/15
8/8 [==============================] - 6s 731ms/step - loss: 0.6222 - f1: 0.6957 - recall: 0.6957 - precision: 0.6957 - accuracy: 0.7004
Epoch 3/15
8/8 [==============================] - 6s 732ms/step - loss: 0.5387 - f1: 0.7204 - recall: 0.7204 - precision: 0.7204 - accuracy: 0.7166
Epoch 4/15
8/8 [==============================] - 6s 730ms/step - loss: 0.5439 - f1: 0.7369 - recall: 0.7369 - precision: 0.7369 - accuracy: 0.7368
Epoch 5/15
8/8 [==============================] - 6s 732ms/step - loss: 0.5254 - f1: 0.7510 - recall: 0.7510 - precision: 0.7510 - accuracy: 0.7530
Epoch 6/15
8/8 [==============================] - 6s 731ms/step - loss: 0.5332 - f1: 0.7354 - recall: 0.7354 - precision: 0.7354 - accuracy: 0.7368
Epoch 7/15
8/8 [==============================] - 6s 730ms/step - loss: 0.4980 - f1: 0.7634 - recall: 0.7634 - precision: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
dense_kfold(inputs= np.array(ycb_x), targets= np.array(ycb_y), batch_size=32, no_epochs=15, num_folds=4, verbosity=1)

1631167763.853515
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 6s 734ms/step - loss: 0.4850 - f1: 0.7525 - recall: 0.7525 - precision: 0.7525 - accuracy: 0.7530
Epoch 2/15
8/8 [==============================] - 6s 735ms/step - loss: 0.4514 - f1: 0.7909 - recall: 0.7909 - precision: 0.7909 - accuracy: 0.7976
Epoch 3/15
8/8 [==============================] - 6s 735ms/step - loss: 0.4640 - f1: 0.7495 - recall: 0.7495 - precision: 0.7495 - accuracy: 0.7530
Epoch 4/15
8/8 [==============================] - 6s 755ms/step - loss: 0.4589 - f1: 0.7768 - recall: 0.7768 - precision: 0.7768 - accuracy: 0.7814
Epoch 5/15
8/8 [==============================] - 6s 731ms/step - loss: 0.4191 - f1: 0.7962 - recall: 0.7962 - precision: 0.7962 - accuracy: 0.7935
Epoch 6/15
8/8 [==============================] - 6s 733ms/step - loss: 0.4087 - f1: 0.8212 - recall: 0.8212 - precision: 0.8212 - accuracy: 0.8178
Epoch 7/15
8/8 [==============================] - 6s 736ms/step - loss: 0.4404 - f1: 0.7823 - recall: 0.7823 - precision: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
grades = {
    "acc": [alex_gray_acc,alex_rgb_acc,alex_hsv_acc,alex_lab_acc,alex_ycb_acc],
    "f1": [alex_gray_f1,alex_rgb_f1,alex_hsv_f1,alex_lab_f1,alex_ycb_f1],
    'sen':[alex_gray_sen,alex_rgb_sen,alex_hsv_sen,alex_lab_sen,alex_ycb_sen]
}
df = pd.DataFrame(grades)
df.index = ["gray", "rgb", "hsv", "lab",'ycb']  #自訂索引值
df.columns = ["accuracy", "f1_score", "sensitivity"]  #自訂欄位名稱
display(df)

,accuracy,f1_score,sensitivity
gray,0.65060,0.62906,0.65148
rgb,0.71084,0.61751,0.61167
hsv,0.66265,0.64400,0.67072
lab,0.72289,0.69761,0.71457
ycb,0.68675,0.62793,0.62551


In [ ]:
grades = {
    "acc": [vgg_gray_acc,vgg_rgb_acc,vgg_hsv_acc,vgg_lab_acc,vgg_ycb_acc],
    "f1": [vgg_gray_f1,vgg_rgb_f1,vgg_hsv_f1,vgg_lab_f1,vgg_ycb_f1],
    'sen':[vgg_gray_sen,vgg_rgb_sen,vgg_hsv_sen,vgg_lab_sen,vgg_ycb_sen]
}
df = pd.DataFrame(grades)
df.index = ["gray", "rgb", "hsv", "lab",'ycb']  #自訂索引值
df.columns = ["accuracy", "f1_score", "sensitivity"]  #自訂欄位名稱
display(df)

,accuracy,f1_score,sensitivity
gray,0.68675,0.40714,0.5
rgb,0.68675,0.40714,0.5
hsv,0.68675,0.40714,0.5
lab,0.68675,0.40714,0.5
ycb,0.68675,0.40714,0.5


In [ ]:
grades = {
    "acc": [res_gray_acc,res_rgb_acc,res_hsv_acc,res_lab_acc,res_ycb_acc],
    "f1": [res_gray_f1,res_rgb_f1,res_hsv_f1,res_lab_f1,res_ycb_f1],
    'sen':[res_gray_sen,res_rgb_sen,res_hsv_sen,res_lab_sen,res_ycb_sen]
}
df = pd.DataFrame(grades)
df.index = ["gray", "rgb", "hsv", "lab",'ycb']  #自訂索引值
df.columns = ["accuracy", "f1_score", "sensitivity"]  #自訂欄位名稱
display(df)

,accuracy,f1_score,sensitivity
gray,0.74699,0.70281,0.70517
rgb,0.49398,0.48125,0.50524
hsv,0.55422,0.46433,0.46377
lab,0.51807,0.49142,0.50234
ycb,0.63855,0.57069,0.57274


In [ ]:
grades = {
    "acc": [ori_gray_acc,ori_rgb_acc,ori_hsv_acc,ori_lab_acc,ori_ycb_acc],
    "f1": [ori_gray_f1,ori_rgb_f1,ori_hsv_f1,ori_lab_f1,ori_ycb_f1],
    'sen':[ori_gray_sen,ori_rgb_sen,ori_hsv_sen,ori_lab_sen,ori_ycb_sen]
}
df = pd.DataFrame(grades)
df.index = ["gray", "rgb", "hsv", "lab",'ycb']  #自訂索引值
df.columns = ["accuracy", "f1_score", "sensitivity"]  #自訂欄位名稱
display(df)

,accuracy,f1_score,sensitivity
gray,0.57831,0.56596,0.59885
rgb,0.65060,0.64037,0.68286
hsv,0.78313,0.75747,0.76889
lab,0.68675,0.40714,0.50000
ycb,0.68675,0.40714,0.50000


In [ ]:
grades = {
    "acc": [dense_gray_acc,dense_rgb_acc,dense_hsv_acc,dense_lab_acc,dense_ycb_acc],
    "f1": [dense_gray_f1,dense_rgb_f1,dense_hsv_f1,dense_lab_f1,dense_ycb_f1],
    'sen':[dense_gray_sen,dense_rgb_sen,dense_hsv_sen,dense_lab_sen,dense_ycb_sen]
}
df = pd.DataFrame(grades)
df.index = ["gray", "rgb", "hsv", "lab",'ycb']  #自訂索引值
df.columns = ["accuracy", "f1_score", "sensitivity"]  #自訂欄位名稱
display(df)

,accuracy,f1_score,sensitivity
gray,0.46988,0.22913,0.34259
rgb,0.98795,0.97801,0.99123
hsv,0.92771,0.89437,0.94542
lab,0.74699,0.69465,0.67446
ycb,0.77108,0.75422,0.79776
